In [ ]:
%cd ..

In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import numpy as np
import torch
from scipy.spatial.transform import Rotation as R

from convert.joints2bvh import Animation, joints2bvh, BVH
from convert.joints2bvh.Quaternions import Quaternions
from serve.inference_worker import (
    # get_abs_data_from_jointpos,
    get_jointpos_from_bvh,
    # unpack_motion,
    # unpacked_motion_to_jointpos,
)

np.set_printoptions(precision=6, suppress=True)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/Users/jason/repos/diffusion-motion-inbetweening/diffusion/nn.py:151: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, run_function, length, *args):
/Users/jason/repos/diffusion-motion-inbetweening/diffusion/nn.py:161: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, *output_grads):


In [2]:
# Load original from file
bvh_path = Path("dataset/HumanML3D/bvh/test/011978_fromjoint100_a_person_walked_forward_by_changing_direcion_into_left_and_right.bvh")
bvh_jointpos, bvh_anim = get_jointpos_from_bvh(bvh_path, return_anim=True)
bvh_anim: Animation.Animation


In [3]:
packed_motion = {0: [[0.0, 0.991798, 0.0], [0.432524, 2.004411, -4.117227], [1.232139, 0.579718, 7.544916], [-0.921, 0.155394, -2.134829], [1.272458, 4.302102, 3.96743], [0.0, 0.0, 0.0], [0.292299, -0.86995, 8.199702], [-1.162997, -0.06573, -3.924885], [-0.132957, -2.780239, 5.572746], [0.0, 0.0, 0.0], [-1.4647, -0.448069, 14.718186], [5.122796, -0.4927309999999999, -26.203743], [-0.144674, -1.8826810000000003, 21.985984], [2.160806, 0.87386, -34.28213], [0.0, 0.0, 0.0], [-31.324777, -6.532802, 2.465906], [-40.884231, -3.000562, -5.521812], [-23.100088, -29.769632, 7.592946999999999], [0.0, 0.0, 0.0], [14.881677, 8.619321, 3.240624], [47.884327000000006, -2.742168, -7.382379], [22.210931, 35.395242, 8.252408], [0.0, 0.0, 0.0]], 1: [[0.00015, 0.992205, -0.000803], [0.420091, 1.805115, -4.12638], [1.223173, 0.582232, 7.529176], [-0.9546850000000001, 0.158478, -2.122266], [1.363459, 4.471889, 3.973531], [0.0, 0.0, 0.0], [0.236095, -0.855694, 8.135654], [-1.081324, -0.071188, -3.9033580000000003], [-0.05667199999999999, -2.633149, 5.59703], [0.0, 0.0, 0.0], [-1.487738, -0.441647, 14.720842], [5.245681, -0.546729, -25.867424], [-0.201714, -1.554368, 21.810366], [2.160147, 0.860339, -33.594344], [0.0, 0.0, 0.0], [-31.675196, -6.460801, 2.308474], [-39.876719, -2.533204, -5.490403], [-24.944138, -30.419957, 8.272847], [0.0, 0.0, 0.0], [14.961302, 8.409345, 3.175071], [47.424331, -2.839509, -7.335282], [24.287908, 35.371175, 8.924992], [0.0, 0.0, 0.0]], 2: [[3.6e-05, 0.992727, -0.001074], [0.388174, 1.683983, -4.191905], [1.250598, 0.591679, 7.6528849999999995], [-0.968218, 0.162425, -2.220871], [1.404342, 4.550525, 3.985478], [0.0, 0.0, 0.0], [0.306747, -0.860661, 8.143077], [-1.131465, -0.065087, -3.854698], [-0.001224, -2.515535, 5.591003], [0.0, 0.0, 0.0], [-1.5189030000000001, -0.4388020000000001, 14.810396000000003], [5.2393, -0.5539830000000001, -25.772151], [-0.13997, -1.440565, 21.820637], [2.194835, 0.814608, -33.134561], [0.0, 0.0, 0.0], [-31.741299, -6.4139740000000005, 2.259746], [-39.362933, -2.175096, -5.524871], [-25.691061, -30.681765000000002, 8.555734], [0.0, 0.0, 0.0], [14.975216999999999, 8.451287, 3.202776], [47.275552, -3.207594, -7.448673], [25.445248, 35.451745, 9.324711], [0.0, 0.0, 0.0]], 3: [[8.4e-05, 0.993038, -0.001516], [0.385429, 1.71539, -4.200146], [1.259189, 0.593227, 7.687165], [-0.990232, 0.166952, -2.325461], [1.384653, 4.513663, 3.9845970000000004], [0.0, 0.0, 0.0], [0.320658, -0.846449, 7.995915], [-1.130862, -0.054463, -3.6425939999999994], [-0.048878, -2.583663, 5.500909], [0.0, 0.0, 0.0], [-1.521583, -0.442429, 14.914098], [5.255723000000001, -0.558918, -25.75223], [-0.156008, -1.528673, 21.810646], [2.203645, 0.768648, -32.736085], [0.0, 0.0, 0.0], [-31.745166999999995, -6.436251, 2.304719], [-38.862309, -1.953371, -5.529196000000001], [-25.831615, -30.417031, 8.508087], [0.0, 0.0, 0.0], [15.026790000000002, 8.501638, 3.213774], [47.177297, -3.459688, -7.521067000000001], [25.685237, 35.218209, 9.320602], [0.0, 0.0, 0.0]], 4: [[7.2e-05, 0.992956, -0.001695], [0.3824020000000001, 1.8575560000000002, -4.219522], [1.297045, 0.5935130000000001, 7.756894], [-1.03248, 0.173456, -2.453429], [1.326116, 4.379975, 3.913996], [0.0, 0.0, 0.0], [0.31889, -0.825284, 7.783092000000001], [-1.068594, -0.039355, -3.242957], [-0.139171, -2.694308, 5.294966], [0.0, 0.0, 0.0], [-1.519059, -0.44841299999999995, 15.000262000000001], [5.329318, -0.576701, -25.72192], [-0.17423400000000003, -1.694621, 21.807901], [2.210712, 0.721389, -32.347531], [0.0, 0.0, 0.0], [-31.709862, -6.508924000000001, 2.389956], [-38.543023, -1.9864260000000002, -5.486235], [-25.305071, -29.71033, 8.140712], [0.0, 0.0, 0.0], [15.184351000000001, 8.464972, 3.181317], [47.158272, -3.565345, -7.554425], [24.895906, 34.7961, 8.931247], [0.0, 0.0, 0.0]], 5: [[-0.00027, 0.992697, -0.001463], [0.37212100000000004, 1.9282840000000003, -4.302269], [1.355211, 0.604401, 7.969807], [-1.055799, 0.18026, -2.654084], [1.278269, 4.291678, 3.9156760000000004], [0.0, 0.0, 0.0], [0.329419, -0.80461, 7.574838], [-0.9923330000000001, -0.019385, -2.704893], [-0.22672900000000004, -2.785754, 5.036519], [0.0, 0.0, 0.0], [-1.485364, -0.45715, 15.085586999999999], [5.390147000000001, -0.593976, -25.677356], [-0.135478, -1.774953, 21.851313], [2.198864, 0.686139, -31.921143999999998], [0.0, 0.0, 0.0], [-31.661885, -6.550881, 2.439797], [-38.508127, -2.126511, -5.451851], [-23.953407000000002, -28.944696, 7.542736], [0.0, 0.0, 0.0], [15.345133, 8.360051, 3.123551], [47.425769, -3.6347130000000005, -7.629867], [23.217066, 34.373946, 8.28425], [0.0, 0.0, 0.0]], 6: [[-0.000867, 0.99231, -0.000851], [0.391298, 1.9742179999999998, -4.430263], [1.368609, 0.61735, 8.158921], [-1.052571, 0.179622, -2.639481], [1.27177, 4.245039000000001, 3.820153], [0.0, 0.0, 0.0], [0.30790300000000004, -0.733173, 6.921136], [-0.898693, 0.044218, -1.197068], [-0.363201, -2.829812, 4.36775], [0.0, 0.0, 0.0], [-1.387167, -0.47451100000000007, 15.203197000000001], [5.430068, -0.602355, -25.698945], [-0.152082, -1.8082650000000002, 21.887986], [2.225328, 0.624597, -31.333177000000003], [0.0, 0.0, 0.0], [-31.592621, -6.434371, 2.364684], [-38.585739000000004, -2.294052, -5.420948], [-22.394291, -28.118226, 6.899417], [0.0, 0.0, 0.0], [15.440237999999999, 8.208402, 3.048403], [47.782137, -3.5999960000000004, -7.684005000000001], [21.307401, 33.740732, 7.536383], [0.0, 0.0, 0.0]], 7: [[-0.001772, 0.991674, 0.000265], [0.5479210000000001, 1.9755180000000003, -4.640923], [1.353403, 0.640067, 8.42718], [-1.248362, 0.193351, -2.509243], [1.300819, 4.195963, 3.543287], [0.0, 0.0, 0.0], [0.262613, -0.622771, 5.919852], [-0.9221630000000001, 0.157496, 1.154172], [-0.6065590000000001, -2.926103, 3.188087], [0.0, 0.0, 0.0], [-1.408093, -0.47550900000000007, 15.331529], [5.475625, -0.618619, -25.644819], [-0.21503000000000003, -1.975252, 21.953177], [2.226812, 0.534445, -30.578429000000003], [0.0, 0.0, 0.0], [-31.610617, -6.295812, 2.348669], [-38.704362, -2.378292, -5.443391], [-20.89356, -27.463352, 6.351273000000001], [0.0, 0.0, 0.0], [15.570095000000002, 8.227147, 3.021483], [47.781654, -3.48972, -7.647203000000001], [20.108365, 33.101026, 7.003899], [0.0, 0.0, 0.0]], 8: [[-0.003697, 0.990621, 0.00174], [0.726668, 1.8053350000000001, -4.947444], [1.439488, 0.674698, 8.914079], [-1.553625, 0.220734, -2.532876], [1.369196, 4.258801, 3.372102], [0.0, 0.0, 0.0], [0.11868699999999999, -0.532883, 5.160823], [-0.751926, 0.233529, 3.300938], [-0.763257, -2.827815, 2.011252], [0.0, 0.0, 0.0], [-1.456524, -0.47547800000000007, 15.646478000000002], [5.484307, -0.624624, -25.658963], [-0.269484, -2.134553, 22.014724], [2.173573, 0.414244, -29.279854], [0.0, 0.0, 0.0], [-31.644115000000003, -6.223009, 2.42221], [-38.677, -2.237061, -5.524178], [-19.913914, -27.095598, 6.022918], [0.0, 0.0, 0.0], [15.641867000000001, 8.333465, 3.014222], [47.648108, -3.503198, -7.631212], [19.761049, 32.906109, 6.830476], [0.0, 0.0, 0.0]], 9: [[-0.007162, 0.989281, 0.004217], [0.748498, 1.4491130000000003, -5.48564], [1.458801, 0.727281, 9.511664], [-1.237545, 0.186619, -2.170613], [1.5966780000000003, 4.566028000000001, 3.027934], [0.0, 0.0, 0.0], [0.361013, -0.526012, 5.002578], [-0.839629, 0.320899, 4.920411], [-0.813863, -2.559643, 1.068748], [0.0, 0.0, 0.0], [-1.348529, -0.495129, 16.049813], [5.542052, -0.658301, -25.480493], [-0.454809, -2.245722, 22.137651], [1.846654, 0.385819, -27.310975], [0.0, 0.0, 0.0], [-31.659566000000005, -5.993791, 2.414557], [-38.34417, -2.012963, -5.601458], [-19.647322, -26.941824000000004, 5.928932], [0.0, 0.0, 0.0], [15.838725, 8.388996, 2.9687660000000005], [47.357476, -3.4643870000000003, -7.57482], [19.959043, 33.040579, 6.890156], [0.0, 0.0, 0.0]], 10: [[-0.011574, 0.987017, 0.006692], [0.808717, 1.169553, -5.888253], [1.732738, 0.741049, 9.905953], [-1.289638, 0.183517, -1.790422], [1.730928, 4.740849, 2.80033], [0.0, 0.0, 0.0], [0.534394, -0.537872, 5.056022], [-0.794185, 0.358968, 5.887602], [-0.837784, -2.29214, 0.325031], [0.0, 0.0, 0.0], [-1.2882150000000003, -0.510889, 16.488461], [5.692553, -0.711909, -25.353184], [-0.652763, -2.49308, 22.257434], [1.150384, 0.518507, -24.518764000000004], [0.0, 0.0, 0.0], [-32.12358, -5.600556, 2.324158], [-37.511805, -1.925815, -5.578233], [-19.845215, -26.930842000000002, 5.980755], [0.0, 0.0, 0.0], [16.757378, 8.228765, 2.7872350000000004], [46.207145, -3.400865, -7.352085], [20.647315, 33.543156, 7.165806], [0.0, 0.0, 0.0]], 11: [[-0.017146, 0.983567, 0.010047], [0.9198870000000001, 0.623653, -6.0272190000000005], [2.016945, 0.730671, 9.911587], [-1.403832, 0.182741, -1.195552], [1.9875969999999998, 5.128988, 2.494144], [0.0, 0.0, 0.0], [0.6884810000000001, -0.510528, 4.7629], [-0.714595, 0.394648, 6.895424], [-0.740507, -1.736895, -0.328468], [0.0, 0.0, 0.0], [-1.238354, -0.504406, 16.502642], [5.812799, -0.785681, -24.925416], [-0.878785, -2.615222, 22.553875], [0.295368, 0.749093, -20.868668], [0.0, 0.0, 0.0], [-32.491665, -5.073904, 2.166079], [-36.945664, -2.340706, -5.468178], [-19.755776, -26.90388, 5.965464], [0.0, 0.0, 0.0], [17.551865, 8.131073, 2.623315], [45.401742, -2.794483, -7.000955], [21.085563, 33.911806, 7.344652000000001], [0.0, 0.0, 0.0]], 12: [[-0.023836, 0.980652, 0.01328], [1.085137, -0.22926400000000002, -6.234123], [2.302613, 0.738322, 10.04367], [-1.57588, 0.190997, -0.61734], [2.349017, 5.737204, 2.193942], [0.0, 0.0, 0.0], [0.875393, -0.478418, 4.427918], [-0.68157, 0.445895, 7.983629], [-0.613433, -0.987178, -1.139796], [0.0, 0.0, 0.0], [-1.154445, -0.504168, 16.883848], [5.788387, -0.841086, -24.321501], [-1.139362, -2.444594, 22.895731000000005], [-0.204456, 0.890231, -18.21589], [0.0, 0.0, 0.0], [-32.960593, -4.665972, 2.048538], [-36.480724, -2.714176, -5.382033], [-18.991403, -27.319355, 5.883645], [0.0, 0.0, 0.0], [18.214386, 8.18197, 2.5544], [45.301724, -2.183452, -6.777385], [20.558138, 34.468904, 7.299618], [0.0, 0.0, 0.0]], 13: [[-0.031667, 0.977771, 0.016781], [1.24025, -1.006518, -6.107179], [2.703223, 0.698633, 9.694049], [-1.779549, 0.199935, 0.22854000000000002], [2.67813, 6.300228, 1.907806], [0.0, 0.0, 0.0], [1.26239, -0.442209, 3.956323], [-0.848407, 0.522341, 8.786162], [-0.534681, -0.370211, -1.8581070000000002], [0.0, 0.0, 0.0], [-1.085901, -0.500933, 17.219319], [5.7199, -0.885907, -23.698635], [-1.362948, -2.383884, 23.347635], [-0.517339, 0.98871, -16.936790000000002], [0.0, 0.0, 0.0], [-33.185053, -4.347888, 2.031364], [-36.566834, -3.058887, -5.389947], [-17.902731, -27.807438, 5.742466], [0.0, 0.0, 0.0], [18.521745, 8.363176, 2.547927], [45.644921, -1.6717900000000003, -6.659089], [19.799745, 34.896873, 7.149347], [0.0, 0.0, 0.0]], 14: [[-0.040028, 0.975524, 0.020321], [1.33776, -1.919621, -5.672059], [3.159632, 0.594868, 8.455731], [-2.005183, 0.198456, 2.231103], [3.168055, 7.095635, 1.245918], [0.0, 0.0, 0.0], [1.515755, -0.39238, 3.409197], [-0.641487, 0.514205, 9.214822], [-0.374672, 0.471409, -2.507654], [0.0, 0.0, 0.0], [-0.918606, -0.501994, 17.356548], [5.675406, -0.9285200000000001, -23.165638], [-1.526734, -2.241111, 23.57338], [-0.9028980000000001, 1.171958, -15.990908000000001], [0.0, 0.0, 0.0], [-33.181287, -4.132275, 2.074613], [-37.274454, -3.359357, -5.468435], [-16.700574, -27.500554000000005, 5.402049], [0.0, 0.0, 0.0], [18.620436, 8.704727, 2.625347], [45.937689, -1.240652, -6.534567000000001], [19.339826, 34.27362, 6.837058], [0.0, 0.0, 0.0]], 15: [[-0.047777, 0.972859, 0.024467], [1.446266, -2.749466, -5.250852], [3.6613209999999996, 0.381913, 5.53867], [-2.504639, 0.225648, 7.573063000000001], [3.8219330000000005, 7.910149999999999, -0.8130280000000001], [0.0, 0.0, 0.0], [1.595648, -0.350876, 2.998881], [-0.218152, 0.460016, 9.545299], [-0.23795000000000002, 1.259509, -3.109304], [0.0, 0.0, 0.0], [-0.659766, -0.515837, 17.402776], [5.653393, -0.9753500000000002, -22.614765], [-1.721327, -2.027817, 23.738316], [-1.207197, 1.347461, -15.717891999999999], [0.0, 0.0, 0.0], [-32.962768, -4.101343, 2.193651], [-38.440148, -4.152826, -5.442666], [-15.654637, -26.564047, 4.980668], [0.0, 0.0, 0.0], [18.508576, 9.175109, 2.805701], [46.498542, -0.4745150000000001, -6.331514], [18.050112, 32.793149, 6.118436], [0.0, 0.0, 0.0]], 16: [[-0.054419, 0.969117, 0.029182], [1.367799, -3.413949, -5.197497], [4.419391, 0.11973700000000001, 1.388144], [-3.034569, 0.350706, 16.165598], [4.264089, 8.267984, -3.243166], [0.0, 0.0, 0.0], [1.621185, -0.333088, 2.832335], [0.564935, 0.36540200000000006, 10.525613], [-0.315185, 1.860501, -4.276332], [0.0, 0.0, 0.0], [-0.288001, -0.568126, 18.008283], [5.736665, -1.050916, -22.146948], [-1.763886, -1.802842, 23.889898], [-1.28677, 1.393949, -15.823683], [0.0, 0.0, 0.0], [-33.316206, -4.072352, 2.247389], [-39.259044, -4.548577, -5.465633], [-14.910672, -26.889219, 4.887788], [0.0, 0.0, 0.0], [18.636167, 9.934512, 3.165946], [46.41489800000001, 0.24397099999999997, -6.058612], [16.498015, 31.053647, 5.32653], [0.0, 0.0, 0.0]], 17: [[-0.059557, 0.965963, 0.034838], [1.277911, -4.100539, -6.096918], [4.841031, -0.11855600000000001, -3.5918540000000005], [-3.143286, 0.556324, 27.734603], [4.732891, 8.31042, -7.3813889999999995], [0.0, 0.0, 0.0], [1.577146, -0.40864600000000006, 3.608669], [0.93983, 0.36186500000000005, 11.869734], [-0.208995, 3.268595, -5.715368], [0.0, 0.0, 0.0], [-0.262336, -0.6236490000000001, 21.391113], [5.850282, -0.873504, -24.862318], [-1.384908, -1.233899, 25.57045], [-1.055099, 1.278276, -16.674518], [0.0, 0.0, 0.0], [-33.310271, -4.517562, 2.598017], [-41.515157, -4.935602, -5.773953], [-12.32728, -27.79276, 4.469096], [0.0, 0.0, 0.0], [19.885646, 11.352116, 4.002472], [45.10525, -0.192878, -6.197642], [15.013014000000002, 30.953607, 4.928963], [0.0, 0.0, 0.0]], 18: [[-0.062547, 0.964082, 0.041539], [1.812834, -4.430377, -5.831873], [4.535937, -0.514454, -11.438552], [-3.980946, 1.328836, 39.963198], [5.171857, 8.780714, -9.485768], [0.0, 0.0, 0.0], [1.168749, -0.362396, 3.349191], [0.806098, 0.436352, 12.705493], [-0.361872, 3.438632, -6.474197], [0.0, 0.0, 0.0], [-0.9171400000000001, -0.495184, 22.112468], [6.170597, -0.951565, -24.878248000000003], [-0.990919, -0.319519, 26.020746], [-0.771867, 1.171809, -16.183741], [0.0, 0.0, 0.0], [-33.490826, -4.5162, 2.317844], [-42.708485, -5.725113, -5.61001], [-11.014477, -29.018820000000005, 4.36074], [0.0, 0.0, 0.0], [20.183546, 11.652263, 4.376744], [45.310677, 0.914109, -5.928382], [12.039744, 29.408157, 3.9574650000000005], [0.0, 0.0, 0.0]], 19: [[-0.063803, 0.964074, 0.051015], [2.857194, -4.462729, -5.60997], [3.133587, -1.117957, -19.60202], [-4.380699, 2.026348, 49.683946], [5.729299, 9.669217, -11.185138], [0.0, 0.0, 0.0], [0.697155, -0.374853, 3.709482], [-0.297079, 0.669929, 12.640943], [-0.534858, 3.302363, -6.944352], [0.0, 0.0, 0.0], [-2.014573, -0.275518, 22.77042], [6.424388, -1.000495, -25.008106], [-0.733253, 0.676807, 26.378653], [-0.45141500000000007, 1.068802, -15.693671], [0.0, 0.0, 0.0], [-33.368844, -4.736102, 2.080398], [-42.888437, -6.081744, -5.418815], [-11.332845, -30.79858, 4.726074], [0.0, 0.0, 0.0], [20.465381, 11.676862, 4.673978], [45.725833, 1.744098, -5.792055], [9.48848, 28.53597, 3.270231], [0.0, 0.0, 0.0]], 20: [[-0.063889, 0.964524, 0.063064], [3.644304, -3.977857, -5.43615], [2.197022, -1.8087529999999998, -28.256003], [-5.535533, 3.051797, 56.968482], [6.51557, 11.394956000000002, -11.926936], [0.0, 0.0, 0.0], [0.949607, -0.516387, 4.998664000000001], [-2.172766, 0.981126, 11.510953], [-0.872725, 2.468618, -7.274931], [0.0, 0.0, 0.0], [-2.603285, -0.169, 23.906718], [6.215445, -0.8989370000000001, -25.488002], [-0.293611, 0.5956460000000001, 26.111839000000003], [-0.599853, 1.114163, -13.711222], [0.0, 0.0, 0.0], [-34.032584, -5.61606, 2.6037380000000003], [-41.962219, -5.188043, -5.558774], [-11.80899, -33.378036, 5.277369], [0.0, 0.0, 0.0], [20.267314, 13.22379, 5.553946], [45.802048, 0.85719, -6.087914], [8.906107, 29.281995, 3.2193750000000003], [0.0, 0.0, 0.0]], 21: [[-0.0635, 0.964365, 0.077693], [4.291346, -3.891627, -5.5483400000000005], [1.8886579999999997, -2.295327, -35.390274000000005], [-7.625449000000001, 4.718555, 59.759171], [7.723457000000001, 14.453206, -12.166233], [0.0, 0.0, 0.0], [-0.246898, -0.5855400000000001, 6.64515], [-1.612718, 0.807974, 10.657249], [-0.9376830000000002, 2.604062, -7.715245], [0.0, 0.0, 0.0], [-3.4144720000000004, 0.012763, 24.800111000000005], [6.371474, -0.9586350000000001, -25.250831], [0.063201, 1.349017, 26.039475], [-0.462686, 1.118759, -13.432218], [0.0, 0.0, 0.0], [-34.622381, -6.517498, 2.878264], [-40.499146, -4.915528, -5.359763], [-12.867676, -34.555637, 5.729308], [0.0, 0.0, 0.0], [20.999516, 13.808136, 6.132646], [44.099338, 0.878747, -5.898003], [10.158734, 30.149063000000005, 3.6540669999999995], [0.0, 0.0, 0.0]], 22: [[-0.064478, 0.962908, 0.096378], [4.942629, -4.233011000000001, -5.649491000000001], [2.080415, -2.461806, -41.01631], [-9.95846, 6.428287000000001, 59.021038], [8.482765, 16.402248, -11.926782], [0.0, 0.0, 0.0], [-0.165053, -0.698218, 7.969103], [-3.048428, 1.095769, 10.653427], [-1.2440460000000002, 2.709206, -8.99293], [0.0, 0.0, 0.0], [-4.867417, 0.365359, 25.185472], [6.527826, -1.017526, -25.086627], [0.632099, 1.9356060000000002, 26.103992], [-0.324106, 1.059077, -13.154953], [0.0, 0.0, 0.0], [-34.434282, -6.962485, 3.049695], [-39.92899, -4.556606, -5.357494], [-13.436908, -34.726963, 5.90162], [0.0, 0.0, 0.0], [21.327539, 14.507055, 6.63703], [42.720625, 1.3878580000000003, -5.572638], [12.444087, 30.685853, 4.312836], [0.0, 0.0, 0.0]], 23: [[-0.066227, 0.959327, 0.120389], [5.644048, -4.211939, -6.309871000000001], [1.679087, -2.766478, -44.044216], [-11.488691, 7.095394, 55.91074100000001], [8.612952, 16.723116, -11.517534], [0.0, 0.0, 0.0], [0.195413, -0.87064, 9.650001], [-4.819482, 1.55975, 11.763769], [-1.892064, 2.182181, -10.962886], [0.0, 0.0, 0.0], [-6.438374, 0.735809, 25.75517], [6.550954, -1.047681, -24.942436], [0.80273, 1.908685, 26.089798], [-0.065185, 0.859734, -13.319786000000002], [0.0, 0.0, 0.0], [-34.16884, -7.204373, 3.266255], [-39.239408, -3.8855289999999996, -5.487518], [-13.656997, -34.475531, 5.899693], [0.0, 0.0, 0.0], [21.498246, 14.910784, 6.833424], [42.354716, 2.363201, -5.209168], [13.672863, 31.860700000000005, 4.833079000000001], [0.0, 0.0, 0.0]], 24: [[-0.066696, 0.954648, 0.14942], [5.651998, -4.342081, -6.944537], [1.051284, -3.078227, -45.419335], [-11.500008, 6.727457000000001, 52.988623], [8.343628, 16.245576, -8.656758], [0.0, 0.0, 0.0], [-0.360531, -1.012293, 12.210162], [-4.355869, 1.47543, 11.88685], [-2.116826, 2.532804, -12.267421], [0.0, 0.0, 0.0], [-6.763049, 0.812979, 25.873744], [6.472949000000001, -1.078552, -24.545824], [0.45775400000000005, 1.962274, 25.961767], [0.0016420000000000002, 0.815596, -13.451599], [0.0, 0.0, 0.0], [-34.682317, -7.376741999999999, 3.39749], [-38.215789, -3.253274, -5.532851], [-13.308213, -32.954729, 5.470022], [0.0, 0.0, 0.0], [21.863893, 15.075738, 6.923814], [42.07516, 3.418998, -4.828277], [13.981272, 33.271948, 5.1701630000000005], [0.0, 0.0, 0.0]], 25: [[-0.068224, 0.950386, 0.182497], [5.118384, -5.766151, -7.134083], [1.638494, -2.532571, -45.025022], [-11.707102, 6.87194, 50.530995], [7.653134, 15.023877, -4.46688], [0.0, 0.0, 0.0], [-0.817603, -1.117211, 15.187733], [-3.569836, 1.198595, 10.50996], [-1.8779530000000002, 4.260535, -12.866545], [0.0, 0.0, 0.0], [-6.887879, 0.875718, 24.923827], [6.241861, -1.099556, -23.925954], [0.734037, 3.079959, 25.934611], [0.077599, 0.847032, -13.749566], [0.0, 0.0, 0.0], [-34.644469, -7.184791000000001, 3.120255], [-38.52451, -2.801231, -5.65648], [-12.114213, -30.413503000000002, 4.668289], [0.0, 0.0, 0.0], [21.64551, 15.233063, 7.07042], [42.462562000000005, 4.245314, -4.613003], [14.000693, 34.392649, 5.406582], [0.0, 0.0, 0.0]], 26: [[-0.07106, 0.948155, 0.219231], [4.733941, -7.804167, -7.056320000000001], [2.347121, -1.7905970000000002, -42.393199], [-12.101136, 7.272516, 47.948245], [6.465672, 12.885311, -1.263755], [0.0, 0.0, 0.0], [-1.140152, -1.1308560000000003, 17.967876], [-3.665811, 1.103287, 9.158381], [-1.218876, 6.366562, -12.211616], [0.0, 0.0, 0.0], [-7.658591, 1.081606, 23.610028], [5.950602, -1.100291, -23.467565], [1.407372, 4.398306, 25.821107], [0.387744, 0.68924, -14.140732], [0.0, 0.0, 0.0], [-34.31323100000001, -6.065437, 2.256566], [-39.488851, -1.636615, -6.09459], [-10.783933, -28.767162, 4.069146], [0.0, 0.0, 0.0], [21.322364, 14.961505, 6.914583], [43.008422, 4.960093, -4.417676], [14.665788, 35.289539, 5.80432], [0.0, 0.0, 0.0]], 27: [[-0.071811, 0.949293, 0.25883], [4.211709, -9.254855, -6.667149], [1.8952280000000001, -1.700092, -39.813317], [-11.867361, 7.609582999999999, 48.285677], [7.421044000000001, 14.636884, -4.024407], [0.0, 0.0, 0.0], [-1.360455, -1.055879, 18.902294], [-3.927697, 1.313573, 10.801223], [-0.232577, 7.933258, -10.306482], [0.0, 0.0, 0.0], [-7.836002, 1.156594, 22.940909], [5.715208, -0.969556, -24.396205], [1.350328, 4.688849, 26.237569], [0.962271, 0.266191, -15.30276], [0.0, 0.0, 0.0], [-34.372091, -4.353576, 1.299618], [-39.697942, -0.083897, -6.68739], [-11.066515000000003, -27.676025, 4.013266], [0.0, 0.0, 0.0], [20.977343, 14.256022000000002, 6.355327], [43.355964, 5.488599, -4.272315], [16.396439, 36.251753, 6.532069], [0.0, 0.0, 0.0]], 28: [[-0.071085, 0.956737, 0.298695], [4.014848, -9.289923, -6.092881], [0.606729, -2.16213, -38.717724], [-10.093773, 6.901161, 50.537437], [7.4264220000000005, 14.627741000000002, -4.961073], [0.0, 0.0, 0.0], [-2.041051, -0.835948, 16.48575], [-3.392708, 1.927891, 17.804408], [1.4038000000000002, 9.122366, -6.477675], [0.0, 0.0, 0.0], [-8.609477, 1.273731, 22.230487], [5.886415, -1.040571, -23.978796], [1.083385, 3.9920260000000005, 25.737814], [1.505342, -0.256723, -15.107686], [0.0, 0.0, 0.0], [-34.511108, -4.14443, 1.442654], [-39.986335, 1.179171, -7.144526], [-10.459474, -26.413891, 3.6891500000000006], [0.0, 0.0, 0.0], [21.268818, 13.392276, 5.682456], [43.384691, 5.275938, -4.242923], [17.134029, 37.785439, 7.052733], [0.0, 0.0, 0.0]], 29: [[-0.070951, 0.966516, 0.34225], [3.453234, -9.228023, -6.78094], [-1.030238, -2.556063, -33.622989], [-8.079477, 5.591322, 49.616085], [7.678676000000001, 15.096134, -8.586998], [0.0, 0.0, 0.0], [-2.761337, -0.490268, 10.504501], [-1.290092, 2.989009, 32.434071], [4.702919, 9.503146, 2.774575], [0.0, 0.0, 0.0], [-9.275556, 1.464163, 23.188207], [6.222123, -1.030426, -25.00791], [1.506466, 3.409048, 26.271174], [1.668456, -0.572728, -13.68131], [0.0, 0.0, 0.0], [-33.084898, -5.713167, 2.882393], [-42.14971400000001, 1.554567, -7.761495000000001], [-10.44236, -24.849148, 3.467923], [0.0, 0.0, 0.0], [20.39967, 13.670807, 5.647938], [44.956601, 4.076585, -4.824873], [16.108097, 39.224756, 7.000449], [0.0, 0.0, 0.0]], 30: [[-0.067773, 0.971827, 0.378548], [3.230241, -9.909484, -6.31751], [-2.013332, -2.696573, -31.808059000000004], [-7.079759, 5.322495, 50.59814], [8.238329, 16.463141, -12.063443], [0.0, 0.0, 0.0], [-4.689035, 0.24996699999999997, -1.8493580000000003], [3.367521, 3.345423, 52.239263], [7.091383, 6.379948, 14.436885], [0.0, 0.0, 0.0], [-9.394428, 1.435762, 22.026355], [6.499203, -1.124615, -24.689675], [1.295082, 3.299608, 26.399289], [1.64604, -0.660478, -13.188405], [0.0, 0.0, 0.0], [-32.313327, -5.158005, 2.734578], [-42.233618, 1.072797, -7.685875], [-12.106781, -24.821401, 3.856141], [0.0, 0.0, 0.0], [20.537152, 12.582116000000001, 4.919085], [44.369786, 4.23163, -4.684167], [16.680706, 39.073632, 7.113791], [0.0, 0.0, 0.0]], 31: [[-0.065037, 0.971814, 0.413977], [3.400931, -10.336220000000003, -7.131901], [-2.691972, -2.682913, -28.765382000000002], [-7.122145, 5.632274, 50.994005], [8.539818, 17.378183, -14.548212], [0.0, 0.0, 0.0], [-5.643695, 0.888225, -12.700859], [6.325745, 3.450861, 69.26967], [5.128926, 4.669848, 11.736997], [0.0, 0.0, 0.0], [-9.359702, 1.500908, 22.753631], [6.719156, -1.207025, -24.592231], [0.702431, 2.72134, 26.755145], [1.173323, -0.360719, -14.255904000000001], [0.0, 0.0, 0.0], [-32.453295, -4.591489, 2.722493], [-40.488021, 0.156006, -7.263903000000001], [-14.129999, -26.400381, 4.621835], [0.0, 0.0, 0.0], [21.746206, 12.264714, 4.517054], [42.619862, 4.666494, -4.325224], [16.537026, 39.192911, 6.983025], [0.0, 0.0, 0.0]], 32: [[-0.06287, 0.97333, 0.448904], [2.758084, -9.54011, -8.137703], [-3.000819, -2.542521, -24.761966], [-6.824033, 5.190395, 50.5065], [8.415447, 17.044812, -16.916842], [0.0, 0.0, 0.0], [-6.644133, 1.479444, -22.361745], [9.201402, 1.064935, 79.200727], [2.771207, 1.622227, 9.042911], [0.0, 0.0, 0.0], [-7.712598, 1.184371, 24.18022], [6.726317, -1.219982, -24.622142], [-0.441416, 1.170752, 26.880664], [0.580483, -0.113566, -13.80524], [0.0, 0.0, 0.0], [-33.57641, -4.880685, 3.390215], [-38.346021, -1.466836, -6.625163], [-13.323491, -27.647638, 4.641756], [0.0, 0.0, 0.0], [23.090476, 12.803449, 4.501256], [41.472336, 4.902043, -4.040424], [14.374780000000001, 38.05703, 5.921021], [0.0, 0.0, 0.0]], 33: [[-0.061268, 0.977056, 0.483345], [1.906954, -8.097604000000002, -8.714658], [-2.256299, -2.131494, -20.864753], [-7.5838790000000005, 5.083005, 49.239935], [8.029792, 15.618098, -18.575865], [0.0, 0.0, 0.0], [-7.815626, 1.778841, -30.083109000000004], [10.398094, -1.251545, 79.860055], [0.094847, -2.609115, 5.872073], [0.0, 0.0, 0.0], [-5.662738, 0.69249, 25.252508], [6.556528, -1.142933, -24.962189000000002], [-1.5878, -0.719542, 26.89714], [-0.000801, 0.11332600000000001, -12.782231], [0.0, 0.0, 0.0], [-33.656244, -5.385293, 4.158637], [-38.336917, -3.679105, -6.139172], [-9.587389, -28.325078000000005, 3.8352999999999997], [0.0, 0.0, 0.0], [23.41669, 13.434604, 4.552662], [42.13922, 4.747707, -4.060674], [11.136017, 36.276533, 4.53363], [0.0, 0.0, 0.0]], 34: [[-0.062768, 0.98328, 0.516712], [0.841718, -6.723128, -8.914636], [-0.688564, -1.549932, -16.891185], [-8.432712, 4.909252, 47.172181], [7.487661000000001, 13.481638, -19.730621], [0.0, 0.0, 0.0], [-8.349876, 1.845673, -34.608525], [10.385623, -2.173589, 71.83666], [-2.14495, -5.568931, 0.303359], [0.0, 0.0, 0.0], [-4.150207, 0.288075, 26.092346], [6.472788, -1.068571, -25.361249], [-2.053707, -2.186443, 26.907046], [-0.449153, 0.339418, -12.511791], [0.0, 0.0, 0.0], [-33.413902, -5.457896, 4.429906], [-38.819416, -6.35224, -5.516835], [-5.658494, -28.912847, 2.959663], [0.0, 0.0, 0.0], [23.940249, 13.531189, 4.421431], [42.462752, 4.043974, -4.255503], [9.419261, 34.847378, 3.791045], [0.0, 0.0, 0.0]], 35: [[-0.066835, 0.990228, 0.548256], [-0.287511, -4.948191, -8.343821], [0.656232, -1.135886, -13.916455], [-8.287215, 4.075598000000001, 45.191591], [6.925917, 10.967743, -20.544247], [0.0, 0.0, 0.0], [-8.737004000000002, 1.636861, -36.553109], [10.154727, -2.646335, 58.502829000000006], [-2.732544, -6.497785, -0.9914340000000001], [0.0, 0.0, 0.0], [-3.283886, -0.033956, 26.141194], [6.607523, -1.032122, -25.776081], [-2.12388, -3.471816, 27.18105], [-0.651822, 0.44916600000000007, -13.247247], [0.0, 0.0, 0.0], [-32.812378, -5.232244, 4.326964], [-39.05048, -9.38335, -4.691637], [-3.470648, -29.580203, 2.510161], [0.0, 0.0, 0.0], [24.271028, 12.845192, 3.982262], [42.550022, 3.514157, -4.439297], [9.324601, 33.780832, 3.632201], [0.0, 0.0, 0.0]], 36: [[-0.07439, 0.99715, 0.58114], [-1.417538, -3.518742, -7.721277999999999], [2.358128, -0.719262, -10.829148], [-8.054638, 3.337764, 42.825869], [6.45547, 8.75426, -20.968717], [0.0, 0.0, 0.0], [-7.688051000000001, 1.750619, -36.287272], [8.745024, -1.9869030000000003, 45.02014], [-1.537916, -4.400442, 1.178159], [0.0, 0.0, 0.0], [-3.091474, -0.187416, 25.52709], [7.05791, -1.158743, -25.250366], [-2.019289, -3.8396790000000003, 27.28613], [-0.390447, 0.372602, -15.490214], [0.0, 0.0, 0.0], [-31.637933, -4.511395, 3.560132], [-38.838377, -11.918896, -3.850875], [-4.423259, -31.185439, 2.949419], [0.0, 0.0, 0.0], [24.335849, 11.203405, 3.21542], [43.08157500000001, 3.500592, -4.574386], [10.040258, 33.235972, 3.8150790000000003], [0.0, 0.0, 0.0]], 37: [[-0.082895, 1.004244, 0.616585], [-2.240281, -1.9951140000000003, -6.956004], [3.180695, -0.442903, -7.447788], [-6.649434, 2.081148, 39.886609], [6.112172000000001, 6.834787000000001, -21.438402], [0.0, 0.0, 0.0], [-6.468837, 1.8306120000000001, -34.593258], [7.130795, -1.3505410000000002, 35.140643], [0.649959, 0.029832, 2.942128], [0.0, 0.0, 0.0], [-3.066721, -0.289572, 24.19046], [7.212086000000001, -1.186095, -25.028496000000004], [-1.705445, -4.3928020000000005, 27.571252], [-0.750396, 0.679954, -15.856376], [0.0, 0.0, 0.0], [-30.834019000000005, -4.538883, 3.42491], [-38.68165, -12.948716, -3.497381], [-5.312507, -33.38303, 3.46603], [0.0, 0.0, 0.0], [24.186853, 10.412483, 2.902642], [43.562511, 2.456294, -5.042708], [11.123385, 33.084379, 4.136952], [0.0, 0.0, 0.0]], 38: [[-0.090394, 1.012761, 0.659013], [-3.192542, -0.317864, -7.14252], [3.835324, -0.081977, -1.786395], [-4.38763, 0.5875, 35.285919], [5.644633, 4.614025, -21.101516], [0.0, 0.0, 0.0], [-5.29803, 1.74924, -30.768161], [7.768689000000001, -1.749451, 30.650555], [0.358012, -1.498163, 4.850137], [0.0, 0.0, 0.0], [-1.9420439999999999, -0.600513, 23.28028], [7.227943, -1.179279, -24.875605000000004], [-1.661204, -4.50439, 27.485092], [-1.173527, 1.107905, -15.415510999999999], [0.0, 0.0, 0.0], [-30.648584999999997, -4.194153, 2.788849], [-38.954658, -12.694793, -3.462349], [-6.801523, -35.816799, 4.229117], [0.0, 0.0, 0.0], [23.889988, 9.372067, 2.641492], [43.963522, 0.814174, -5.756399000000001], [12.019365, 32.971468, 4.444912], [0.0, 0.0, 0.0]], 39: [[-0.095793, 1.021716, 0.708742], [-3.6484559999999995, 0.760492, -8.229363], [4.223023, 0.333826, 6.038317], [-2.330536, -0.342608, 29.281281], [4.92019, 2.299533, -19.769135], [0.0, 0.0, 0.0], [-3.255699, 1.699361, -23.829749], [6.487169, -1.300401, 27.06563], [-0.213421, -3.282935, 6.299093], [0.0, 0.0, 0.0], [-0.475911, -0.894602, 22.211782], [6.964097, -1.125158, -24.690131], [-1.605901, -3.857232, 27.568295], [-1.49482, 1.5134170000000002, -14.016652], [0.0, 0.0, 0.0], [-30.373796, -3.121437, 1.502616], [-40.480007, -11.122543, -3.8636750000000006], [-9.417645, -38.222014, 5.46987], [0.0, 0.0, 0.0], [22.727207, 8.230526, 2.487998], [44.746721, -1.074646, -6.70003], [13.136384000000001, 32.751506, 4.833735], [0.0, 0.0, 0.0]], 40: [[-0.100093, 1.028695, 0.758747], [-3.137776, 1.320075, -8.599164], [4.796562000000001, 0.544745, 11.613331], [-2.204163, -0.34162, 25.968379], [3.777923, -0.35993, -17.613875], [0.0, 0.0, 0.0], [-2.092429, 1.475768, -18.290304], [4.656143, -0.66805, 24.665358], [-0.9028859999999999, -3.817875, 3.7612859999999997], [0.0, 0.0, 0.0], [0.306052, -0.915631, 19.642841000000004], [6.609433, -1.099651, -23.750451], [-1.447391, -2.65143, 27.791254], [-1.328403, 1.582582, -12.90619], [0.0, 0.0, 0.0], [-30.140889999999995, -1.647529, -0.092572], [-41.8234, -9.105428, -4.419063], [-13.091799, -39.308178, 6.942125], [0.0, 0.0, 0.0], [22.070697, 6.865862000000001, 2.219345], [44.429541, -2.174257, -7.187456000000001], [14.426439, 32.078737, 5.177518], [0.0, 0.0, 0.0]], 41: [[-0.104018, 1.032784, 0.800521], [-2.340574, 1.796009, -8.164606], [5.399188, 0.42821400000000004, 10.479288000000002], [-3.413154, -0.070351, 31.433685999999998], [2.073966, -2.942559, -13.637116], [0.0, 0.0, 0.0], [-2.873184, 1.365023, -18.704214], [5.046557, -0.9027, 29.327167000000003], [-1.4541440000000003, -3.9961249999999997, 0.844484], [0.0, 0.0, 0.0], [0.505113, -0.898395, 18.611202], [6.682067, -1.107614, -23.701031], [-1.550686, -2.024061, 27.678768000000005], [-1.27337, 1.6582350000000001, -12.670303], [0.0, 0.0, 0.0], [-30.270547, 0.217837, -1.724136], [-43.191949, -7.019408000000001, -5.091534], [-15.038695000000002, -39.182308, 7.620855], [0.0, 0.0, 0.0], [21.681213, 6.168579, 2.092525], [43.808226, -0.689576, -6.651580000000001], [14.059507, 31.029484, 4.929005], [0.0, 0.0, 0.0]], 42: [[-0.104572, 1.032003, 0.833211], [-1.538396, 3.173291, -8.390406], [6.394134, 0.102612, 3.338098], [-5.217005, 0.19829, 46.627431], [-0.29943, -5.499378000000001, -7.930059000000001], [0.0, 0.0, 0.0], [-3.7030190000000003, 1.077866, -17.051234], [5.121868, -1.12585, 31.377338], [-2.496401, -5.142890000000001, -3.243483], [0.0, 0.0, 0.0], [1.417472, -1.153278, 19.980653], [6.774443, -1.088523, -23.949416], [-2.353484, -2.035288, 27.334405000000004], [-2.037567, 2.369887, -13.133908], [0.0, 0.0, 0.0], [-30.840343000000004, -0.278975, -1.7912689999999998], [-43.012622, -4.691906, -5.71236], [-15.519062000000002, -38.41216, 7.520323], [0.0, 0.0, 0.0], [21.543978, 6.912037, 2.741241], [42.82947300000001, 0.9721470000000001, -6.007086], [12.136097, 32.324795, 4.655963], [0.0, 0.0, 0.0]], 43: [[-0.103947, 1.026847, 0.860762], [-0.845942, 5.462046, -8.93959], [7.945485000000001, -0.141867, -4.711175], [-7.095034, 0.291449, 60.85181], [-2.183232, -6.029472, -2.0718410000000005], [0.0, 0.0, 0.0], [-5.358286, 0.6521990000000001, -15.027196999999997], [6.163492, -1.9394710000000002, 33.366914], [-3.401395, -6.798942000000001, -6.195056], [0.0, 0.0, 0.0], [1.8791330000000002, -1.40412, 21.344983], [7.098491000000001, -1.209348, -23.078261], [-2.868432, -1.9844920000000001, 27.098589], [-2.183537, 2.645586, -12.839188], [0.0, 0.0, 0.0], [-31.935858, -2.631193, -0.886262], [-41.381437, -3.835834, -5.58226], [-15.314311, -35.651512, 6.656155], [0.0, 0.0, 0.0], [21.824434, 7.988236, 3.751516], [40.743082, 2.100159, -5.479523], [10.405928, 34.920643, 4.60834], [0.0, 0.0, 0.0]], 44: [[-0.100335, 1.022702, 0.885507], [0.115408, 6.52276, -8.794964], [9.481833, -0.24214700000000003, -11.939726], [-9.029658, 1.241852, 69.437708], [-1.27926, -4.402039, -2.776108], [0.0, 0.0, 0.0], [-6.818302, 0.395582, -14.792471000000003], [6.31118, -2.323767, 36.442146], [-3.87535, -7.351792999999999, -9.190675], [0.0, 0.0, 0.0], [1.70107, -1.4889050000000001, 22.339186], [7.2055299999999995, -1.252529, -22.68237], [-2.822177, -1.093622, 26.889458], [-1.7969060000000001, 2.512903, -12.089315], [0.0, 0.0, 0.0], [-33.423013, -4.407991, -0.445583], [-39.928233, -3.2139540000000006, -5.346211], [-14.577258, -32.325233, 5.6183], [0.0, 0.0, 0.0], [22.137578000000005, 9.21784, 4.879826], [38.472479, 3.726156, -4.836082], [8.252037, 36.118667, 4.141023], [0.0, 0.0, 0.0]], 45: [[-0.093131, 1.019734, 0.908775], [1.257426, 7.301099, -8.100545], [11.604937, -0.166197, -18.350909], [-10.478373, 2.269039, 71.300318], [-0.680791, -3.1441560000000006, -2.979789], [0.0, 0.0, 0.0], [-8.868356, 0.067852, -14.502477000000003], [6.769302, -2.779161, 38.407493], [-4.16929, -7.501752, -11.87913], [0.0, 0.0, 0.0], [1.256037, -1.464396, 22.719512], [7.184951, -1.23215, -22.587315], [-2.849684, -0.598344, 26.888453], [-1.713172, 2.54951, -11.853605], [0.0, 0.0, 0.0], [-34.162546, -4.677413, -0.734639], [-39.853207, -2.1906740000000005, -5.502452], [-13.732349, -30.369499000000005, 4.973331], [0.0, 0.0, 0.0], [22.101925, 9.957697000000001, 5.558051], [37.113523, 5.616748, -4.235757], [5.77898, 36.869525, 3.470062], [0.0, 0.0, 0.0]], 46: [[-0.083691, 1.017687, 0.932048], [2.721828, 7.524185, -7.156772], [13.96241, 0.129698, -22.378967], [-11.096499, 2.8039200000000006, 65.728331], [0.515515, -1.285687, -4.53883], [0.0, 0.0, 0.0], [-11.37164, -0.292541, -14.155067], [7.174488, -3.074921, 39.741057], [-4.298876, -7.052396, -14.321983], [0.0, 0.0, 0.0], [0.272131, -1.283288, 22.793136], [6.926332, -1.157914, -22.45097], [-2.800736, -0.11955200000000002, 26.930721], [-1.641209, 2.589566, -12.309784], [0.0, 0.0, 0.0], [-34.47857, -4.226858, -1.333539], [-39.615209, -0.661017, -5.843995], [-13.439405, -29.563888, 4.730479], [0.0, 0.0, 0.0], [22.449901, 9.872059, 5.698675000000001], [36.33904, 7.131717, -3.8120010000000004], [4.067348, 37.981129, 3.06609], [0.0, 0.0, 0.0]], 47: [[-0.070713, 1.016921, 0.957702], [4.199124, 7.289051000000001, -6.678095], [15.772939000000001, 0.454235, -22.816198], [-9.909209, 2.039079, 53.756434], [1.4955430000000003, 0.20940800000000004, -5.884624], [0.0, 0.0, 0.0], [-13.571507, -0.5576100000000002, -13.134109000000002], [6.774059, -2.833626, 40.685095], [-4.381039000000001, -6.305795, -16.568618], [0.0, 0.0, 0.0], [-0.848754, -1.059196, 23.028051], [6.510481, -1.064932, -22.177841], [-2.704957, 0.266063, 26.970341], [-1.720279, 2.701753, -12.736346], [0.0, 0.0, 0.0], [-34.579229, -3.870527, -1.6626310000000002], [-39.10004, 0.611409, -6.119106], [-13.303306, -28.626339, 4.523817], [0.0, 0.0, 0.0], [22.831155, 9.782077, 5.723075], [36.019287, 8.092187, -3.557], [3.361854, 38.771232, 2.923083], [0.0, 0.0, 0.0]], 48: [[-0.053455, 1.015425, 0.984592], [5.213834, 7.045404, -6.22609], [17.26502, 0.660542, -20.920624000000004], [-7.432825, 0.9036480000000001, 37.852858], [1.494648, 0.24197299999999997, -5.596093], [0.0, 0.0, 0.0], [-15.469188, -0.736191, -12.007986], [6.224975, -2.511623, 41.507917], [-4.450864, -5.579762, -18.415275], [0.0, 0.0, 0.0], [-1.308288, -0.9411859999999999, 22.922257], [5.986412, -0.93965, -21.893225], [-2.882083, 0.187491, 27.002216000000004], [-2.033936, 2.934127, -13.335068], [0.0, 0.0, 0.0], [-34.455673, -3.39539, -1.8557550000000003], [-38.92145500000001, 1.396136, -6.357599], [-12.727969, -27.771382, 4.248069], [0.0, 0.0, 0.0], [22.73392, 10.015113, 5.767756], [36.278311, 8.666874, -3.394054], [3.749655, 38.780991, 3.043426], [0.0, 0.0, 0.0]], 49: [[-0.031584, 1.012607, 1.012554], [5.55128, 6.388691, -5.770395], [18.300132, 0.735781, -18.184344], [-4.784425, 0.262518, 23.178171], [1.278591, 0.8588880000000001, -3.547864], [0.0, 0.0, 0.0], [-16.980616, -0.795336, -10.817941], [5.64791, -2.079181, 41.884952], [-4.481208, -4.605421000000001, -19.990171000000004], [0.0, 0.0, 0.0], [-1.106639, -0.9215300000000001, 22.545751], [5.484463, -0.816639, -21.71979], [-3.095014, 0.044905, 26.964556], [-2.511758, 3.2490360000000003, -13.975896], [0.0, 0.0, 0.0], [-34.393047, -2.807851, -2.025724], [-38.815123, 2.110872, -6.609959], [-12.396549, -27.163211, 4.091535], [0.0, 0.0, 0.0], [22.587831, 10.368966, 5.801782], [36.315341, 8.589798, -3.3566530000000006], [5.630989, 38.597711, 3.608524], [0.0, 0.0, 0.0]], 50: [[-0.005086, 1.008271, 1.041687], [5.172793, 4.759441, -5.404381], [20.197343, 0.931137, -15.593258], [-5.588386, 0.657444, 14.32296], [0.253875, 1.1658, 1.253411], [0.0, 0.0, 0.0], [-18.025262, -0.676711, -9.272337], [4.855588, -1.31941, 41.114549], [-4.443759, -3.137016, -21.081447], [0.0, 0.0, 0.0], [-0.650337, -0.9140380000000001, 22.098089], [5.229075, -0.79702, -21.457039], [-2.959952, 0.357826, 26.679768], [-3.119172, 3.64819, -14.525626], [0.0, 0.0, 0.0], [-34.525504, -2.182735, -2.23028], [-38.678094, 2.88072, -6.851377], [-12.667132, -26.841662, 4.126289], [0.0, 0.0, 0.0], [22.619045, 10.603331, 5.747905], [35.908343, 8.130751, -3.370204], [8.119578, 38.216537, 4.328925], [0.0, 0.0, 0.0]], 51: [[0.025666, 1.004157, 1.072718], [4.376229, 2.527427, -5.019452], [21.723163, 1.085756, -13.287167], [-8.31685, 1.285255, 11.597932], [-0.375174, 1.9740879999999998, 5.889133], [0.0, 0.0, 0.0], [-18.929158, -0.432125, -7.253834], [3.7856440000000005, -0.367856, 38.936657], [-4.243559, -1.12653, -21.272793], [0.0, 0.0, 0.0], [-0.510014, -0.827143, 21.531707], [5.356844, -0.888306, -21.31792], [-2.5073740000000004, 0.9916110000000001, 26.387619000000004], [-3.620624, 3.95796, -14.935738], [0.0, 0.0, 0.0], [-34.445616, -1.362277, -2.532832], [-38.744762, 3.673646, -7.143147], [-13.995545, -27.126163, 4.524423], [0.0, 0.0, 0.0], [22.863549, 10.384094, 5.432699], [35.620195, 7.440673, -3.450589], [10.177485, 37.48519, 4.847977], [0.0, 0.0, 0.0]], 52: [[0.059297, 1.001062, 1.1045], [2.965326, 0.706363, -4.838726], [22.29733, 1.323437, -13.471973000000002], [-9.621171, 1.859393, 16.308849000000002], [0.689436, 3.2411860000000003, 4.270383], [0.0, 0.0, 0.0], [-19.206243, -0.117864, -4.504056], [2.206097, 0.575865, 35.615399], [-3.7138260000000005, 0.861868, -19.816751], [0.0, 0.0, 0.0], [-0.670007, -0.729383, 21.241202], [5.885748, -1.055554, -21.549668], [-1.896242, 0.86576, 26.821485], [-3.347198, 3.618972, -15.351852000000001], [0.0, 0.0, 0.0], [-34.313496, -0.62681, -2.477232], [-38.811108, 3.221613, -7.215022000000001], [-15.527879000000002, -27.354129, 5.005361], [0.0, 0.0, 0.0], [23.365919, 9.880942, 4.819138], [35.571033, 6.809793, -3.548319], [10.930543, 36.793164, 4.916681], [0.0, 0.0, 0.0]], 53: [[0.089909, 1.006119, 1.132634], [2.278638, -1.161959, -4.336645], [22.187664, 1.71119, -15.746136000000002], [-11.048986, 2.88473, 23.96342], [1.670434, 4.12381, 1.616435], [0.0, 0.0, 0.0], [-19.46953, 0.124614, -3.237264], [-0.24576800000000001, 1.761028, 33.531285], [-2.463726, 3.606234, -16.14724], [0.0, 0.0, 0.0], [-3.106023, -0.217967, 21.203544], [6.700956, -1.32582, -21.378336], [-0.575702, 1.141944, 26.704495], [-2.86054, 3.145921, -14.86396], [0.0, 0.0, 0.0], [-33.775636, -1.378333, -1.477446], [-38.47774, 1.860682, -6.870114], [-15.750541000000002, -27.366009, 5.083268], [0.0, 0.0, 0.0], [24.924036, 8.920714, 4.01003], [36.554958000000006, 4.989866, -4.022579], [10.145252, 37.019958, 4.681476], [0.0, 0.0, 0.0]], 54: [[0.120876, 1.014128, 1.159899], [0.415283, -2.6129010000000004, -3.320821], [21.093888, 1.566262, -14.993407000000001], [-9.684201, 2.63496, 25.127014], [3.244424, 6.2263910000000005, -1.9370020000000001], [0.0, 0.0, 0.0], [-19.866067, 0.078422, -5.82291], [0.356067, 1.866731, 37.015723], [-0.276924, 6.047156, -9.679915], [0.0, 0.0, 0.0], [-3.066663, -0.178653, 21.120261], [7.225402, -1.4072470000000001, -22.168477], [-0.6273180000000002, 0.077218, 26.806079], [-2.002106, 2.230503, -14.540791], [0.0, 0.0, 0.0], [-33.20658, -1.450483, -0.534512], [-37.98708200000001, 0.188944, -6.554612], [-15.192957, -27.556365, 5.044726], [0.0, 0.0, 0.0], [25.256579, 7.795519, 2.6915400000000003], [38.326561, 3.417393, -4.451526], [11.042302, 36.826885, 4.865278], [0.0, 0.0, 0.0]], 55: [[0.146907, 1.023938, 1.182389], [-1.354039, -5.181364, -2.267505], [21.215135, 1.645891, -13.638471], [-9.724824, 2.911262, 25.200892], [4.422385, 8.521102, -3.298492], [0.0, 0.0, 0.0], [-18.462362, -0.087097, -13.176176], [1.073205, 2.736561, 47.667882], [0.45786400000000005, 5.555376, -6.139005], [0.0, 0.0, 0.0], [-3.342083, -0.023225, 21.855428], [7.839676, -1.525093, -22.86979], [-0.413296, -1.071739, 26.485677], [-1.832138, 1.737471, -14.516548000000002], [0.0, 0.0, 0.0], [-32.761379, -1.49428, 0.655652], [-36.876115, -1.386038, -6.228858], [-14.552036999999999, -28.111143, 5.057776], [0.0, 0.0, 0.0], [25.319712, 7.622276, 1.631489], [39.236464, 1.922084, -4.666548], [13.261275, 37.101644, 5.427093], [0.0, 0.0, 0.0]], 56: [[0.165325, 1.029547, 1.20285], [-1.587318, -8.616745, -1.5973], [21.167214, 1.6094370000000002, -10.537523], [-11.504394, 3.678693, 22.850414000000004], [5.632477, 11.335206, -3.6034780000000004], [0.0, 0.0, 0.0], [-16.77182, 0.085634, -22.163721], [3.827179, 3.20808, 61.67816299999999], [-2.702373, -0.87801, -11.099955], [0.0, 0.0, 0.0], [-4.46036, 0.380189, 22.202149], [7.716098000000001, -1.497409, -23.178943], [-0.9851500000000001, -2.067457, 26.731899], [-2.132025, 1.549975, -15.301177], [0.0, 0.0, 0.0], [-31.639859, -2.060486, 2.380044], [-34.21006, -3.238234, -5.702418], [-13.42954, -27.805119, 4.724806000000001], [0.0, 0.0, 0.0], [26.056212000000002, 8.063221, 0.703298], [40.295949, 0.472949, -4.87242], [15.560538, 37.443142, 5.981143000000001], [0.0, 0.0, 0.0]], 57: [[0.180659, 1.03668, 1.224942], [-2.472616, -10.931513, -1.750117], [20.657741, 1.381304, -6.916219], [-11.307054, 3.6059689999999995, 20.921304], [6.320033, 13.054472, -3.530859], [0.0, 0.0, 0.0], [-14.511657000000001, 0.667642, -27.454697], [9.241391, 0.274932, 67.151756], [-5.697476, -10.701494, -12.8086], [0.0, 0.0, 0.0], [-3.421717, 0.317359, 23.688364], [7.534709000000001, -1.438221, -23.659622], [-2.525899, -4.369973, 26.405668], [-3.136407, 1.800375, -14.249319], [0.0, 0.0, 0.0], [-31.895293000000002, -2.827158, 4.408279], [-31.773581, -5.23514, -5.270427], [-8.990018, -25.220231, 3.207528], [0.0, 0.0, 0.0], [26.111888, 8.751189, -0.299798], [40.952565, -1.32376, -5.052032], [17.917807, 37.051205, 6.308588], [0.0, 0.0, 0.0]], 58: [[0.191084, 1.041697, 1.244912], [-3.30213, -13.175327, -0.728579], [20.327633, 1.290412, -4.82359], [-11.304911, 3.599588, 19.168735], [7.0888, 14.977625000000002, -3.250001], [0.0, 0.0, 0.0], [-11.769821, 1.664705, -31.804516], [13.193684, -2.091971, 62.727959], [-6.127262, -12.351656, -12.158333], [0.0, 0.0, 0.0], [-2.525113, 0.23642999999999997, 23.668024], [7.373986, -1.406587, -23.739983], [-3.537486, -6.414908, 26.31746], [-4.069854, 2.059471, -13.873777], [0.0, 0.0, 0.0], [-31.628164, -3.3576390000000003, 6.139506], [-31.018809, -8.258937, -4.847921], [-3.136097, -21.565673, 1.592096], [0.0, 0.0, 0.0], [25.171095, 8.855184, -1.413805], [42.043705, -3.119436, -5.418593], [19.20467, 37.304608, 6.378866], [0.0, 0.0, 0.0]], 59: [[0.196954, 1.044603, 1.264774], [-3.940488, -15.323142, 0.640783], [19.948869, 1.208986, -3.005203], [-11.384709000000003, 3.5413010000000003, 17.040994], [7.875304, 16.930169, -2.795839], [0.0, 0.0, 0.0], [-8.110431, 2.980879, -34.230079], [12.354964, -1.162927, 51.602604], [-4.413933000000001, -6.731062, -10.214230000000002], [0.0, 0.0, 0.0], [-2.1376, 0.264254, 23.636154], [7.264705, -1.33662, -24.202661], [-3.877089, -8.086192, 26.232889], [-4.928097, 2.29177, -14.240226], [0.0, 0.0, 0.0], [-31.205839, -3.172131, 7.139507], [-30.648816000000004, -11.355558, -4.420383000000001], [0.730526, -19.144226, 0.789052], [0.0, 0.0, 0.0], [24.293533, 8.45271, -2.6292], [42.369886, -4.854128, -5.734105], [19.84399, 36.854408, 6.11772], [0.0, 0.0, 0.0]], 60: [[0.198823, 1.04501, 1.286447], [-4.423588, -16.117081, 1.701878], [19.648415, 1.069788, -1.42797], [-11.256263, 3.344078, 15.326857], [8.17681, 17.744677, -2.750372], [0.0, 0.0, 0.0], [-4.893475, 3.887718, -34.454115], [8.409402, 0.293779, 38.706781], [-1.879577, -1.314325, -5.44101], [0.0, 0.0, 0.0], [-1.425855, 0.162543, 24.067253], [7.3135140000000005, -1.268072, -24.837076], [-4.263545, -9.803634, 26.09957], [-5.899905, 2.674989, -14.861373999999998], [0.0, 0.0, 0.0], [-30.846105, -3.138895, 7.904547000000001], [-29.777893000000002, -13.912777, -3.9963549999999994], [2.401107, -19.778293000000005, 0.566789], [0.0, 0.0, 0.0], [23.677158, 7.7272110000000005, -3.739079], [42.353294, -6.453463, -6.045472], [19.549517, 35.988991, 5.596531], [0.0, 0.0, 0.0]], 61: [[0.195057, 1.042449, 1.312852], [-4.650902, -14.951518, 1.7535400000000003], [19.437352, 0.8223420000000001, 0.320081], [-10.375475, 2.920563, 14.767316999999998], [7.796941, 16.964411, -3.47713], [0.0, 0.0, 0.0], [-2.481264, 4.132032, -33.048991], [3.856495, 1.298482, 30.633934], [-0.15013, 1.013604, -1.857709], [0.0, 0.0, 0.0], [-0.42441600000000007, -0.12149, 24.593069], [7.545703, -1.262925, -25.230821000000002], [-4.684354, -11.643978, 26.079671000000005], [-6.607712, 3.006655, -15.120193], [0.0, 0.0, 0.0], [-30.191293, -4.012179, 8.79207], [-29.098402, -15.730406, -3.630733], [2.794659, -23.083772, 0.624057], [0.0, 0.0, 0.0], [23.261981, 7.447503, -4.201729], [42.24163, -8.733647, -6.696557], [18.517715, 35.038543, 5.025914], [0.0, 0.0, 0.0]], 62: [[0.185732, 1.037783, 1.3431], [-4.295993, -13.080496, 1.268589], [18.865881, 0.504678, 2.696075], [-8.902306, 2.299608, 14.092231], [7.212835, 15.611177, -4.306664], [0.0, 0.0, 0.0], [-1.816233, 3.7715250000000005, -30.940540000000002], [2.670271, 1.355296, 28.413241], [0.257782, 0.399304, 0.380476], [0.0, 0.0, 0.0], [0.306319, -0.398271, 24.347272], [7.700998, -1.256478, -25.45605], [-4.930389, -12.738404, 26.229623], [-6.645456, 3.0283, -14.87863], [0.0, 0.0, 0.0], [-29.494512999999998, -4.829754, 9.191852], [-29.482391, -17.649972, -3.214752], [2.775352, -27.178145, 0.8379820000000001], [0.0, 0.0, 0.0], [22.970298, 6.891265, -4.425697], [41.933106, -11.140115, -7.481007], [18.122648, 33.473347, 4.69463], [0.0, 0.0, 0.0]], 63: [[0.171019, 1.033478, 1.379026], [-3.305584, -11.656771, 0.301244], [17.893872, 0.208123, 6.799018], [-7.407886, 1.65085, 11.960617], [6.690141, 14.376817000000003, -4.531846], [0.0, 0.0, 0.0], [-1.828528, 3.100021, -27.303554], [2.979026, 1.168069, 28.587711000000002], [0.238975, 0.752983, -0.174361], [0.0, 0.0, 0.0], [0.55132, -0.524901, 23.229547], [7.5871770000000005, -1.239143, -25.362915000000005], [-4.853865, -12.620861, 26.310798], [-6.323361000000001, 2.928258, -14.468365000000002], [0.0, 0.0, 0.0], [-29.242536, -4.983514, 8.897087000000003], [-31.417788000000005, -19.439717, -2.776796], [2.747167, -30.191157, 1.093586], [0.0, 0.0, 0.0], [22.561879, 6.043951, -4.432394], [42.154047, -13.100504, -8.365167], [17.190115, 31.414646000000005, 4.254531], [0.0, 0.0, 0.0]], 64: [[0.150635, 1.030855, 1.417973], [-2.233791, -11.155657, -0.875017], [17.46495, 0.058731, 11.580462], [-6.256105, 1.228286, 10.000218], [6.06428, 12.978154, -3.865813], [0.0, 0.0, 0.0], [-1.813589, 2.630206, -24.202848], [3.024234, 1.30137, 30.671248], [-0.821258, -0.013224, -3.105027], [0.0, 0.0, 0.0], [0.976795, -0.617032, 21.397159], [7.174253, -1.245341, -24.571087], [-4.831317, -11.567953, 26.529461], [-6.063437, 2.86352, -14.849283000000002], [0.0, 0.0, 0.0], [-29.753821000000002, -4.01922, 7.887019], [-33.873569, -20.173745, -2.557086], [2.372765, -31.986121, 1.427772], [0.0, 0.0, 0.0], [22.403008, 5.449471, -4.250402], [42.406814, -13.937544, -8.90154], [15.860408, 29.409493, 3.784934], [0.0, 0.0, 0.0]], 65: [[0.126411, 1.028847, 1.457368], [-1.455514, -10.513141, -2.171153], [18.309437, -0.075193, 14.104346], [-5.982295, 1.234747, 12.222815], [4.895287000000001, 10.322601, -2.77354], [0.0, 0.0, 0.0], [-1.714421, 2.369502, -22.368445], [4.104404, 1.186593, 35.109932], [-2.144288, -0.876533, -7.333905], [0.0, 0.0, 0.0], [1.584251, -0.728978, 20.356102], [7.100198, -1.301901, -23.963351], [-5.397697, -11.397069, 26.499585], [-6.054517, 2.884213, -15.320707999999998], [0.0, 0.0, 0.0], [-31.080204999999996, -2.634259, 6.960366], [-35.537374, -18.48793, -3.056347], [1.553803, -32.950277, 1.814205], [0.0, 0.0, 0.0], [22.436717, 5.662991, -3.945526], [42.361608, -12.707126, -8.521297], [14.121351, 27.649875, 3.2249150000000006], [0.0, 0.0, 0.0]], 66: [[0.101346, 1.026814, 1.492501], [-0.570833, -9.791045, -3.415458], [19.40179, 0.012639, 10.424485], [-5.33253, 1.658325, 24.433515], [3.218111, 6.624897, -1.561674], [0.0, 0.0, 0.0], [-0.8281170000000001, 1.9890370000000002, -18.607005], [3.625229, 1.383895, 35.918841], [-3.382362, -1.891785, -11.415047], [0.0, 0.0, 0.0], [1.710465, -0.758595, 20.292374], [7.007627, -1.301223, -23.825359], [-6.011059, -11.518113, 26.426437], [-5.956719, 2.848683, -15.295795], [0.0, 0.0, 0.0], [-31.949714, -2.494415, 6.869444], [-36.107892, -15.187688, -3.974232], [-0.6488840000000001, -32.446815, 2.353506], [0.0, 0.0, 0.0], [22.521776, 6.687196, -3.279566], [41.880500000000005, -10.935381, -7.812366999999999], [13.759721000000003, 28.288088, 3.248093], [0.0, 0.0, 0.0]], 67: [[0.07963, 1.021486, 1.519864], [0.41355, -8.587413, -4.237295], [17.788027, 0.445613, 1.606246], [-1.147679, 0.615312, 42.765951], [2.402221, 3.301793, -5.832965], [0.0, 0.0, 0.0], [-1.4245050000000001, 1.670465, -16.821408], [4.946018, 0.822954, 38.29758600000001], [-4.240918, -3.204176, -14.162085000000001], [0.0, 0.0, 0.0], [1.385107, -0.748392, 21.285112], [7.094329, -1.280714, -24.149816], [-5.992095, -10.498886, 26.575994], [-4.846152, 2.226563, -15.756342999999998], [0.0, 0.0, 0.0], [-32.505237, -3.083584, 6.675361], [-36.08406, -12.861778, -4.460639], [-3.246985, -30.853143, 2.791929], [0.0, 0.0, 0.0], [22.616499, 7.61224, -2.148148], [41.296307, -8.392421, -7.029107], [12.944896, 30.829304, 3.481317], [0.0, 0.0, 0.0]], 68: [[0.063256, 1.017159, 1.545446], [0.8449850000000001, -8.934449, -4.912349], [15.711739999999999, 0.798515, -6.554206], [0.379607, 0.365065, 56.469402], [3.143738, 2.925877, -11.338238], [0.0, 0.0, 0.0], [-1.646644, 1.376378, -14.362386], [5.785787, 0.559934, 38.856932], [-4.623285, -2.910548, -16.270968], [0.0, 0.0, 0.0], [1.7965899999999997, -0.817122, 22.346478], [7.131451, -1.349685, -23.761213], [-5.008413, -7.144992, 26.53164], [-3.789543, 1.957256, -15.494368000000001], [0.0, 0.0, 0.0], [-33.983883000000006, -4.048082, 6.219053000000001], [-35.985621, -10.868924, -4.706968], [-4.703127, -28.824862000000003, 2.804457], [0.0, 0.0, 0.0], [22.496124, 9.307827, -0.035964], [39.746271, -4.848428, -5.961705], [11.065744, 32.853521, 3.473919], [0.0, 0.0, 0.0]], 69: [[0.04972, 1.01606, 1.573209], [1.165433, -10.478914, -5.539941], [13.142427, 0.8957199999999998, -13.297878], [-0.877141, 2.242995, 61.882743], [4.046502, 4.806614, -12.838531], [0.0, 0.0, 0.0], [-1.684999, 1.05531, -11.228341], [5.743665, 0.814907, 38.194083], [-4.665865, -1.139906, -17.746578], [0.0, 0.0, 0.0], [2.251931, -0.834835, 23.484606], [6.9663960000000005, -1.368273, -23.533472], [-3.380687, -2.711432, 26.345868], [-2.963921, 1.955786, -14.544345000000002], [0.0, 0.0, 0.0], [-35.16193, -3.9763389999999994, 4.649886], [-36.795792, -7.981968, -5.150962], [-6.068843, -27.636722000000002, 2.935628], [0.0, 0.0, 0.0], [21.934273, 10.903895, 2.128517], [37.943903, -0.535332, -4.800916000000001], [8.036797, 33.158433, 2.92652], [0.0, 0.0, 0.0]], 70: [[0.038832, 1.016835, 1.601151], [1.365902, -11.357922, -5.362598000000001], [10.14101, 0.629589, -19.533814], [-3.388107, 4.29417, 59.212129000000004], [3.702488, 3.969819, -12.760795000000002], [0.0, 0.0, 0.0], [-2.00337, 0.797624, -8.738782], [5.937172, 0.805169, 37.197903], [-4.655613, 0.412279, -18.818212], [0.0, 0.0, 0.0], [2.811542, -0.9052320000000001, 24.188607], [7.021572, -1.358946, -23.830773], [-2.128129, 0.773071, 26.205479], [-2.2904170000000006, 1.9382939999999997, -14.509742999999999], [0.0, 0.0, 0.0], [-35.951643, -2.909116, 2.4480170000000006], [-37.391349, -4.305284, -5.824438], [-7.801675000000001, -27.258659, 3.266394], [0.0, 0.0, 0.0], [21.515063, 12.033919, 3.7312750000000006], [35.881432, 4.106732000000001, -3.579423], [4.559059, 32.584463, 2.092074], [0.0, 0.0, 0.0]], 71: [[0.029095, 1.0172, 1.633742], [2.284381, -12.623446, -5.50688], [6.278684, -0.095687, -22.489061], [-6.642289, 5.692235, 49.212216000000005], [3.087621, 3.408151, -10.849238], [0.0, 0.0, 0.0], [-2.799301, 0.529754, -5.864516], [5.308958, 1.1949840000000003, 36.242519], [-4.672406, 2.329887, -19.950976], [0.0, 0.0, 0.0], [2.615603, -0.782842, 24.709174000000004], [6.740926, -1.289728, -24.029987], [-1.2124490000000003, 3.789844, 26.073686], [-1.584646, 1.794808, -14.759934000000001], [0.0, 0.0, 0.0], [-35.963738, -1.786816, 0.49694], [-37.331655, -0.295443, -6.577634], [-9.422648, -26.648670000000003, 3.516733], [0.0, 0.0, 0.0], [21.006545, 12.815559, 4.837447], [34.619297, 8.763756, -2.490487], [1.465786, 31.907494000000003, 1.3310140000000001], [0.0, 0.0, 0.0]], 72: [[0.018271, 1.01559, 1.669775], [2.88414, -14.263748, -5.637861], [2.9220870000000003, -0.800247, -23.031385], [-8.739213, 5.157971, 35.674759], [3.594697, 6.385587, -6.102395], [0.0, 0.0, 0.0], [-1.9611390000000002, 0.292601, -2.812916], [2.742579, 2.364344, 35.195609], [-5.012241, 3.380415, -21.230854], [0.0, 0.0, 0.0], [2.121948, -0.575538, 24.96723], [6.404841, -1.217947, -24.206722], [-0.312473, 6.199895, 26.080632], [-1.085313, 1.702042, -15.421247], [0.0, 0.0, 0.0], [-35.445633, -0.731606, -0.9533510000000001], [-37.245938, 3.098999, -7.272185], [-10.386799, -25.111562, 3.473925], [0.0, 0.0, 0.0], [20.366486, 13.554482, 5.618656], [33.931659, 13.078988, -1.511172], [-0.670459, 32.400535, 0.843898], [0.0, 0.0, 0.0]], 73: [[0.004657, 1.012464, 1.710177], [2.5008780000000006, -15.617294999999999, -5.792079], [0.946539, -1.159069, -21.141082], [-8.563769, 3.282119, 21.934577], [4.512738, 9.604397, 2.800041], [0.0, 0.0, 0.0], [-0.4855630000000001, -0.081471, 1.503323], [0.434874, 3.111677, 32.952207], [-5.090219, 4.660844, -21.787514], [0.0, 0.0, 0.0], [2.074535, -0.490006, 25.013833], [6.102706, -1.165229, -24.258009], [0.35942700000000005, 7.604295, 25.988814], [-0.9242980000000001, 1.712924, -15.486748], [0.0, 0.0, 0.0], [-34.96583, -0.016604, -1.750037], [-37.232052, 5.902204, -7.922172], [-10.224576, -22.999119, 3.108284], [0.0, 0.0, 0.0], [19.712463000000003, 14.344802000000001, 6.160731], [33.770406, 15.993402999999999, -0.8196180000000001], [-1.261542, 35.246915, 0.786196], [0.0, 0.0, 0.0]], 74: [[-0.011686, 1.006969, 1.757213], [1.537339, -16.675513, -5.815876], [0.487436, -1.110089, -17.533662], [-7.720888999999999, 2.002329, 11.871211], [6.167718, 12.798123, 8.008257], [0.0, 0.0, 0.0], [0.053508, -0.449039, 6.497006], [0.37418700000000005, 2.799901, 30.31451], [-4.722511, 6.829591, -21.750243], [0.0, 0.0, 0.0], [2.06274, -0.463546, 23.854058], [5.90754, -1.165022, -23.972806], [0.772341, 8.231177, 26.019755000000004], [-0.525486, 1.404044, -14.835049], [0.0, 0.0, 0.0], [-34.126169, 0.620694, -2.109889], [-37.671704, 8.120090000000001, -8.614243], [-9.810986, -20.823754, 2.722497], [0.0, 0.0, 0.0], [18.867937, 14.443592, 6.130708], [34.882784, 17.485463, -0.411013], [0.18588500000000002, 39.571352, 1.419025], [0.0, 0.0, 0.0]], 75: [[-0.031476, 0.999253, 1.809557], [0.480856, -17.103493, -6.031284], [0.732748, -0.952714, -14.8454], [-6.812582, 1.8070150000000003, 12.281568], [8.400042, 16.605802, 9.82543], [0.0, 0.0, 0.0], [0.677852, -0.714295, 10.109861], [1.327755, 2.583525, 31.250731], [-3.805928, 7.91251, -19.841996], [0.0, 0.0, 0.0], [1.105463, -0.301366, 22.511156], [6.087442, -1.196973, -24.117919], [1.135374, 7.919361, 26.260022], [0.371853, 0.572643, -13.597663], [0.0, 0.0, 0.0], [-32.970944, 1.196688, -2.100687], [-38.018561, 9.679863, -9.2023], [-10.228894, -19.184802, 2.578692], [0.0, 0.0, 0.0], [18.222027, 14.120748, 5.753437], [37.094646, 17.255013, -0.391648], [3.438288, 43.544189, 2.832287], [0.0, 0.0, 0.0]], 76: [[-0.051322, 0.989628, 1.86043], [0.437996, -16.751367, -6.326706], [0.671626, -0.9398120000000001, -14.131207], [-7.330419, 2.486492, 17.89583], [8.644214, 17.684375, 3.721636], [0.0, 0.0, 0.0], [-0.36291900000000005, -0.386647, 6.065072], [5.112804, 2.8969, 45.05603500000001], [-0.711954, 8.85291, -12.075388], [0.0, 0.0, 0.0], [-1.48378, 0.115801, 21.180922], [6.55287, -1.297913, -24.087928], [1.17747, 6.676579, 26.63388], [1.567917, -0.58858, -12.797383000000002], [0.0, 0.0, 0.0], [-32.86607, 1.395235, -1.692224], [-37.184021, 9.286883, -9.034592], [-10.844631, -17.64747, 2.460128], [0.0, 0.0, 0.0], [19.278886, 13.267317, 5.05706], [38.74843, 16.133214, -0.626331], [7.263803000000001, 45.587781, 4.466241], [0.0, 0.0, 0.0]], 77: [[-0.067926, 0.979015, 1.90064], [0.920357, -16.011405, -6.436807], [1.543572, -0.899289, -16.542595], [-8.7053, 4.129809, 27.188869], [8.337359, 17.571253, -2.454016], [0.0, 0.0, 0.0], [-1.223117, 0.512343, -5.424214], [10.307983, 2.328523, 66.39463], [1.687171, 5.678214, -1.774307], [0.0, 0.0, 0.0], [-3.937866, 0.550806, 21.210613], [6.838192, -1.377851, -23.86013], [0.653097, 5.442205, 26.602834], [2.214548, -1.232273, -12.766499], [0.0, 0.0, 0.0], [-32.900694, 0.538738, -0.721911], [-36.137732, 7.105889, -8.312275], [-10.530453, -18.197133000000004, 2.483296], [0.0, 0.0, 0.0], [20.747784, 12.470103, 4.431642000000001], [39.796214, 14.610358000000002, -0.9972359999999999], [9.651863, 45.825754, 5.412113], [0.0, 0.0, 0.0]], 78: [[-0.082282, 0.966654, 1.939684], [1.327862, -15.28609, -7.830648], [2.392794, -0.70137, -14.554119], [-10.05452, 5.285781, 31.118879], [8.087115, 17.424362, -7.820788], [0.0, 0.0, 0.0], [-2.786142, 1.475718, -15.537587999999998], [13.461999, 0.43339800000000006, 83.358431], [3.9889880000000004, 11.501072, -1.251393], [0.0, 0.0, 0.0], [-4.887649, 0.80644, 22.892143], [6.7489, -1.4095080000000002, -23.581657], [-0.506609, 3.688985, 26.832122], [1.886114, -1.170593, -13.133], [0.0, 0.0, 0.0], [-33.385955, -0.722655, 0.738841], [-34.79589, 3.7409509999999995, -7.389771], [-10.016216, -20.116028000000004, 2.666204], [0.0, 0.0, 0.0], [22.25119, 12.840838, 4.252532], [39.714528, 13.207414000000002, -1.30313], [10.870369, 43.370651, 5.407558000000001], [0.0, 0.0, 0.0]], 79: [[-0.095148, 0.957954, 1.977178], [0.365501, -13.028511, -8.006549], [3.299524, -0.578675, -13.813098], [-9.165629, 5.12636, 35.399688], [7.497788, 15.735835000000002, -12.107495], [0.0, 0.0, 0.0], [-4.807534, 2.399018, -26.261468], [14.909371000000002, -3.3536910000000004, 91.120926], [2.532119, 7.189702, -0.844669], [0.0, 0.0, 0.0], [-3.499034, 0.45933, 24.201717], [6.8659810000000006, -1.377954, -23.93213], [-2.070989, 1.082721, 26.783438], [1.429945, -1.059087, -12.784316], [0.0, 0.0, 0.0], [-34.340846, -1.553486, 1.825982], [-33.732434, -0.657368, -6.309217], [-8.149426, -22.985019, 2.711378], [0.0, 0.0, 0.0], [23.599461, 12.412715, 3.621381], [39.713773, 10.874584, -1.8639319999999997], [10.94933, 39.372091, 4.770129], [0.0, 0.0, 0.0]], 80: [[-0.109867, 0.953857, 2.015534], [-0.607745, -10.946513, -8.272111], [3.6531049999999996, -0.496193, -11.867399], [-7.174849, 4.037074, 37.984911], [7.112383, 14.177206, -15.433612000000002], [0.0, 0.0, 0.0], [-7.780247, 2.516759, -34.794206], [14.80682, -5.108042, 88.169359], [-2.678044, -5.478574, -3.7147030000000005], [0.0, 0.0, 0.0], [-1.9797560000000003, 0.057436, 25.743838], [6.71641, -1.248353, -24.656911], [-2.852051, -0.711197, 27.071545], [1.059687, -0.862465, -13.379088], [0.0, 0.0, 0.0], [-34.599847, -2.367396, 2.619611], [-33.213195, -5.343754, -5.23198], [-6.752553000000001, -27.07278, 2.935292], [0.0, 0.0, 0.0], [24.600218, 11.773039, 3.086686], [40.045126, 7.332313999999999, -2.918896], [10.114187, 35.932353, 4.039362], [0.0, 0.0, 0.0]], 81: [[-0.12686, 0.956117, 2.055212], [-2.123869, -9.604074, -7.873518000000001], [5.160915, -0.212227, -8.642074], [-5.50908, 2.797732, 37.189674], [6.697026, 12.517323, -17.037073], [0.0, 0.0, 0.0], [-9.73445, 2.225798, -39.643591], [12.776204, -3.476829, 74.546039], [-5.34049, -11.999045, -6.258805], [0.0, 0.0, 0.0], [-0.190301, -0.430162, 26.713898], [6.694445, -1.17302, -25.178786], [-2.754912, -1.827995, 27.023453], [0.226449, -0.252848, -13.878713], [0.0, 0.0, 0.0], [-34.380001, -2.905016, 3.062967], [-33.934237, -9.534404, -4.332129], [-5.846879, -31.028603, 3.231611], [0.0, 0.0, 0.0], [25.284827, 11.338356, 2.747352], [40.23294, 3.678511, -4.012486], [8.687937, 32.402724, 3.220039], [0.0, 0.0, 0.0]], 82: [[-0.146115, 0.963865, 2.095953], [-3.4050300000000004, -8.580363, -6.628374], [6.807484, 0.026766, -4.767428], [-4.173546, 1.6850350000000003, 33.45753400000001], [6.282633, 11.052148, -17.375904], [0.0, 0.0, 0.0], [-10.595159, 1.821912, -39.953153], [8.944818, -1.007215, 53.681096], [-3.9713170000000004, -9.236101, -2.378286], [0.0, 0.0, 0.0], [1.199631, -0.834419, 26.694079], [6.958426, -1.142822, -25.772854], [-2.390583, -2.875249, 26.907332], [-0.989085, 0.625649, -15.071865000000003], [0.0, 0.0, 0.0], [-33.421837, -2.914063, 3.202267], [-36.041731, -12.698042, -3.698199], [-5.722538, -34.236347, 3.7164420000000002], [0.0, 0.0, 0.0], [24.985893, 10.371975, 2.060402], [41.281117, 0.848146, -4.971922000000001], [7.492539, 29.134892, 2.571859], [0.0, 0.0, 0.0]], 83: [[-0.169148, 0.974624, 2.144115], [-4.444815, -7.130368, -5.636617], [8.866522, 0.207124, 1.1519], [-3.3054550000000003, 0.821274, 26.879507], [5.611075, 8.91974, -16.262843], [0.0, 0.0, 0.0], [-10.044784, 1.521415, -35.907514], [4.848333, 0.232025, 34.000273], [-0.222089, -2.039245, 2.643618], [0.0, 0.0, 0.0], [2.591419, -1.22493, 25.431222], [7.309112999999999, -1.199424, -25.647077], [-2.245851, -4.343179, 26.697733], [-2.480621, 1.619061, -15.710129], [0.0, 0.0, 0.0], [-32.44835, -3.018782, 3.497213], [-37.933001, -13.725478, -3.52999], [-7.259687, -36.465519, 4.639061], [0.0, 0.0, 0.0], [24.350192, 9.202764, 1.193544], [42.409603, -2.583986, -6.163732], [7.574547000000001, 27.722744000000002, 2.428242], [0.0, 0.0, 0.0]], 84: [[-0.19341, 0.984255, 2.201573], [-4.502343, -4.851818, -5.751269], [10.022728, 0.23136200000000004, 8.725223], [-1.681692, -0.00918, 19.441553], [4.558346, 5.802717, -14.127307], [0.0, 0.0, 0.0], [-10.635461, 0.821219, -30.078713], [5.163466, -0.335758, 25.06692], [0.520436, -1.687214, 5.457325], [0.0, 0.0, 0.0], [3.676781, -1.4771100000000001, 23.323486], [7.430794, -1.241161, -25.192315], [-2.6150700000000002, -6.144819, 26.370851], [-3.455508, 2.240509, -14.683531000000002], [0.0, 0.0, 0.0], [-31.579308, -3.162091, 3.731627], [-39.319517, -13.177121, -3.7372519999999994], [-10.121844, -36.701906, 5.70481], [0.0, 0.0, 0.0], [23.585311, 7.527163, 0.11597600000000001], [42.826133, -7.011455, -7.639302], [9.521945, 27.981761, 2.862907], [0.0, 0.0, 0.0]], 85: [[-0.219326, 0.985877, 2.261841], [-3.878549, -1.947635, -6.034597], [11.397324, -0.010875, 14.303094000000002], [-0.449237, -0.405497, 15.524017], [3.1936080000000002, 1.9738880000000003, -12.15138], [0.0, 0.0, 0.0], [-11.666209000000002, 0.221327, -25.810448], [7.897215000000001, -1.481792, 25.99952], [-0.1291, -3.1352880000000005, 5.8738], [0.0, 0.0, 0.0], [4.266802, -1.529151, 20.645211], [7.521447000000001, -1.251309, -24.876488], [-3.481566, -7.756770000000001, 26.360117], [-3.8841319999999997, 2.5339400000000003, -14.045618], [0.0, 0.0, 0.0], [-30.915414000000002, -3.092535, 3.608784], [-39.867059, -11.801959, -4.130284], [-13.030243, -35.896504, 6.532233000000001], [0.0, 0.0, 0.0], [23.16763, 5.835807000000001, -0.785816], [42.481264, -10.854915, -8.835699], [11.766809, 28.44033, 3.4282980000000003], [0.0, 0.0, 0.0]], 86: [[-0.248354, 0.98567, 2.319022], [-3.860878, -1.188623, -7.528201000000001], [13.913978, -0.42021, 17.511659], [0.185262, -0.725016, 18.68077], [1.4428500000000002, -1.300847, -8.974669], [0.0, 0.0, 0.0], [-10.133032, 0.414873, -23.937830000000005], [8.276433, -1.8893820000000001, 32.885292], [-1.312237, -3.827966, 1.195518], [0.0, 0.0, 0.0], [4.993369, -1.586381, 19.81604], [7.667523999999999, -1.429327, -23.494453], [-3.9279610000000003, -7.748501, 26.341477], [-4.287273, 2.907497, -14.242795], [0.0, 0.0, 0.0], [-32.18086, -2.783459, 3.2216520000000006], [-38.750442, -10.074747, -4.550544], [-13.086107, -34.456217, 6.160385], [0.0, 0.0, 0.0], [24.008107, 6.223938, -0.489277], [41.008407, -11.238065, -8.6206], [11.362266, 26.836139, 3.156594], [0.0, 0.0, 0.0]], 87: [[-0.272903, 0.988607, 2.369904], [-3.211187, -1.646988, -9.120422], [14.541002999999998, -0.30142500000000005, 14.397089999999999], [0.669418, -1.448522, 31.865369999999995], [-0.127203, -2.260566, -3.820777], [0.0, 0.0, 0.0], [-8.568106, 0.622736, -20.682254000000004], [7.780471000000001, -1.675653, 36.350308], [-2.356585, -4.055289, -4.278491000000001], [0.0, 0.0, 0.0], [4.849606, -1.572654, 20.627143], [7.745412000000001, -1.552877, -22.726267], [-3.9680810000000006, -6.777071, 26.413358], [-4.836861, 3.472106, -13.71076], [0.0, 0.0, 0.0], [-33.337053000000004, -2.608299, 2.83325], [-39.248615, -8.405667, -5.045308], [-10.275204, -31.573199, 4.723904], [0.0, 0.0, 0.0], [24.199794, 8.024798, 0.797353], [41.384668, -8.128484, -7.813963], [6.716843, 23.722309, 1.916664], [0.0, 0.0, 0.0]], 88: [[-0.287454, 0.994948, 2.409153], [-2.442257, -2.204706, -10.019365], [13.088877, 0.09277200000000001, 6.700636], [1.2083820000000003, -2.424086, 48.255543], [0.134578, -1.214168, -3.072058], [0.0, 0.0, 0.0], [-8.803279, 0.455615, -16.273047], [8.228219, -1.724848, 35.149075], [-2.426841, -3.232664, -6.190104], [0.0, 0.0, 0.0], [5.493771, -1.758649, 22.264921], [7.360790000000001, -1.525896, -22.197413], [-3.7678169999999995, -4.24308, 26.19543], [-4.887855, 3.918441, -12.30352], [0.0, 0.0, 0.0], [-33.780787, -4.117763, 2.914035], [-39.964699, -6.860235000000001, -5.280984], [-9.405709, -29.082439999999995, 3.9604410000000003], [0.0, 0.0, 0.0], [23.200641, 9.99394, 2.773131], [42.238326, -4.759497, -7.167648], [2.683358, 23.488144, 1.220364], [0.0, 0.0, 0.0]], 89: [[-0.298335, 1.003731, 2.444109], [-1.631514, -2.21396, -9.986705], [12.240647, 0.251392, -2.028833], [-0.687438, -1.555146, 58.737168], [2.253297, 2.667154, -5.342677], [0.0, 0.0, 0.0], [-9.431549, 0.284443, -12.565383], [8.541623, -1.781482, 32.860021], [-2.408342, -2.533979, -7.309761], [0.0, 0.0, 0.0], [4.662106, -1.65486, 22.95473], [7.357128, -1.566282, -21.767069], [-2.774073, -1.579495, 26.592934000000003], [-4.036186, 3.639716, -11.937928000000001], [0.0, 0.0, 0.0], [-33.944242, -5.321225, 2.655222], [-40.24936, -5.431042, -5.441807], [-11.58473, -27.099633, 4.089414], [0.0, 0.0, 0.0], [22.856409, 10.967695, 4.179209], [39.99940200000001, -0.9485460000000001, -5.886758], [2.4615560000000003, 24.685723, 1.300131], [0.0, 0.0, 0.0]], 90: [[-0.304242, 1.016357, 2.476205], [-0.666047, -2.518566, -9.02664], [11.559261, 0.301411, -10.484658], [-3.49116, 0.736836, 60.861850999999994], [4.519336, 8.524684, -3.491973], [0.0, 0.0, 0.0], [-9.5016, 0.237777, -9.161605], [7.339903, -1.203702, 28.806550000000005], [-2.233534, -1.919971, -7.343667000000001], [0.0, 0.0, 0.0], [3.7089419999999995, -1.498982, 23.40405], [7.088263, -1.452698, -22.119438], [-1.89528, 0.432084, 26.705399], [-3.969131, 3.8500590000000003, -11.566913], [0.0, 0.0, 0.0], [-34.553149, -4.572288, 1.324072], [-40.370969, -3.495657, -5.817844], [-13.778388, -25.524245, 4.291086], [0.0, 0.0, 0.0], [22.240586, 11.608224, 5.070839], [37.186083, 3.08796, -4.517821], [3.730004, 26.681922000000004, 1.710452], [0.0, 0.0, 0.0]], 91: [[-0.306763, 1.027942, 2.50811], [0.309738, -2.237192, -7.835510000000001], [11.369276000000001, 0.314055, -16.918776], [-6.087595, 2.413427, 54.781718], [6.25953, 12.478397, -2.5852330000000006], [0.0, 0.0, 0.0], [-9.354079, 0.19011300000000003, -5.267505], [5.422909, -0.5669640000000001, 23.452579], [-2.126254, -1.847677, -6.939573], [0.0, 0.0, 0.0], [2.644055, -1.314395, 23.366468], [6.672352, -1.2895, -22.430573], [-1.28811, 2.043923, 26.809499], [-3.832798, 3.974263, -11.50108], [0.0, 0.0, 0.0], [-34.546061, -3.998565, 0.131179], [-40.865391, -1.215611, -6.3485], [-14.547913, -24.544274, 4.246355], [0.0, 0.0, 0.0], [21.309851, 11.658291000000002, 5.560992], [35.800794, 6.296418, -3.6682929999999994], [4.425721, 30.159831, 2.195564], [0.0, 0.0, 0.0]], 92: [[-0.306676, 1.037358, 2.542372], [1.47946, -1.705286, -7.04318], [11.138191, 0.262676, -20.076845], [-7.914237000000001, 2.811428, 42.352771], [6.439068, 12.890223, -2.275413], [0.0, 0.0, 0.0], [-10.039813, 0.12175599999999999, -0.569416], [4.053882000000001, -0.257359, 17.69827], [-1.9182600000000003, -1.719395, -6.172022], [0.0, 0.0, 0.0], [1.44136, -1.100119, 23.289537], [5.916248, -1.063646, -22.450248], [-0.996555, 3.511083, 26.750227], [-3.442732, 3.899773, -11.842405], [0.0, 0.0, 0.0], [-34.557111, -3.5994150000000005, -0.9683569999999999], [-40.646111, 0.786336, -6.672559], [-13.991755, -23.16126, 3.822045], [0.0, 0.0, 0.0], [20.618909, 11.286616, 5.786263], [35.64409, 8.645453, -3.170461], [4.49873, 33.702816, 2.627551], [0.0, 0.0, 0.0]], 93: [[-0.303027, 1.042444, 2.579815], [2.365872, -1.179828, -6.450444], [11.105621, 0.217475, -20.75286], [-8.05573, 2.052319, 27.15899], [4.48457, 9.202496, -0.445113], [0.0, 0.0, 0.0], [-11.304629, 0.14357500000000004, 4.130448], [3.904608, -0.303916, 12.547349], [-1.756031, -1.568123, -5.690556], [0.0, 0.0, 0.0], [0.544465, -0.933457, 22.952680000000004], [5.131245, -0.860771, -22.140139], [-0.773968, 4.83669, 26.433707], [-3.089295, 3.8388610000000005, -12.079158], [0.0, 0.0, 0.0], [-34.775812, -3.305776, -1.9964780000000002], [-40.203165, 2.327886, -6.795176], [-12.514619, -21.607225, 3.216928], [0.0, 0.0, 0.0], [19.981387, 11.273732, 6.141447], [35.873776, 10.450522, -2.786712], [4.386498, 36.111343, 2.936724], [0.0, 0.0, 0.0]], 94: [[-0.295507, 1.041717, 2.620643], [2.431142, -1.304024, -6.292371], [11.709624, 0.32625, -19.948682], [-6.731026000000001, 1.181396, 14.633616999999997], [3.297462, 7.471968, 2.051395], [0.0, 0.0, 0.0], [-12.245353, 0.252266, 8.258187], [4.214132, -0.3928110000000001, 9.395597], [-1.590158, -1.017728, -5.777457], [0.0, 0.0, 0.0], [0.377406, -0.877557, 22.456467000000004], [4.428459, -0.7101350000000001, -21.562239], [-0.344991, 6.359001000000001, 26.133206], [-2.763237, 3.789419, -12.098148], [0.0, 0.0, 0.0], [-34.960798, -3.1267640000000005, -2.885149], [-40.122935, 3.8622179999999995, -6.998527], [-10.959866, -20.645913, 2.745803], [0.0, 0.0, 0.0], [19.296378, 11.976662, 6.801538], [35.89705, 11.764823, -2.479238], [4.519831, 37.271025, 3.177101], [0.0, 0.0, 0.0]], 95: [[-0.284338, 1.035403, 2.664491], [1.8253320000000002, -2.007794, -6.554254], [12.684863, 0.529059, -18.694617], [-4.987638, 0.688431, 8.816462], [3.4617170000000006, 8.401978, 4.53119], [0.0, 0.0, 0.0], [-12.96988, 0.41891, 11.683143], [4.495726, -0.438005, 8.386619], [-1.351662, 0.12188500000000001, -6.35707], [0.0, 0.0, 0.0], [0.358434, -0.839507, 22.203666], [4.179787, -0.667491, -21.408884], [0.47036900000000004, 7.822272999999999, 26.059675], [-2.467446, 3.7333700000000003, -12.209192], [0.0, 0.0, 0.0], [-34.911735, -2.516324, -3.849385], [-40.379202, 5.183875, -7.292118], [-10.380252, -19.917111, 2.51959], [0.0, 0.0, 0.0], [19.035513, 12.965383, 7.545636999999999], [35.189784, 12.931308, -2.1841700000000004], [5.041147, 37.110205, 3.3387250000000006], [0.0, 0.0, 0.0]], 96: [[-0.270476, 1.026102, 2.712256], [0.837293, -2.89542, -7.084953], [14.207763, 0.811627, -17.088289], [-6.048785, 0.921814, 9.010368], [4.442338, 10.507319, 6.214196], [0.0, 0.0, 0.0], [-13.169516000000002, 0.5826000000000001, 15.402274], [3.899878, -0.327432, 7.798511000000001], [-1.087348, 1.520909, -7.100168], [0.0, 0.0, 0.0], [-0.011028, -0.725335, 21.490731], [4.38398, -0.770518, -21.142678], [1.611723, 9.025016, 26.323610000000002], [-1.975695, 3.5162810000000007, -12.721617], [0.0, 0.0, 0.0], [-34.201232, -1.857453, -4.516885], [-40.977311, 6.465217, -7.754287], [-10.925956, -20.02342, 2.611279], [0.0, 0.0, 0.0], [19.12836, 13.372261, 7.943532], [34.921052, 14.054157, -1.9371539999999998], [5.314363, 36.075592, 3.316311], [0.0, 0.0, 0.0]], 97: [[-0.250493, 1.016381, 2.763923], [0.278165, -2.661497, -7.132271], [13.902696, 0.701648, -15.930632000000001], [-7.256836999999999, 1.246511, 11.555228], [3.828373, 9.748185, 7.697072000000001], [0.0, 0.0, 0.0], [-14.144099, 0.844292, 18.994814], [3.292433, -0.237659, 7.2732079999999995], [-0.643647, 2.538568, -6.677864], [0.0, 0.0, 0.0], [-0.707681, -0.58708, 20.039666], [4.897931, -0.8922280000000001, -21.339976], [1.962993, 8.892298, 26.514503], [-1.199956, 2.956257, -13.227739], [0.0, 0.0, 0.0], [-33.310079, -0.905922, -4.998164], [-41.024240000000006, 7.5862870000000004, -8.159636], [-12.517564, -20.491835, 2.9580460000000004], [0.0, 0.0, 0.0], [19.623492, 12.487114, 7.561704], [35.880374, 13.865849, -2.036686], [5.874513, 35.869058, 3.507313], [0.0, 0.0, 0.0]], 98: [[-0.224775, 1.010186, 2.815107], [0.584709, -1.554912, -6.743026], [11.381645, 0.266209, -18.094141], [-7.114346, 1.417059, 18.71792], [3.74607, 9.568041, 7.218152], [0.0, 0.0, 0.0], [-15.748605, 1.0761890000000003, 19.626635], [1.962388, 0.079676, 11.076927], [0.208944, 3.502801, -4.61575], [0.0, 0.0, 0.0], [-2.031158, -0.39778, 19.333381], [5.456591, -0.925821, -22.551119], [1.701412, 7.861043, 26.813932], [-0.4913410000000001, 2.426543, -14.388862], [0.0, 0.0, 0.0], [-33.434106, -0.238137, -5.285696], [-40.073844, 6.785172, -7.778465000000001], [-12.895036000000001, -19.483305, 2.881139], [0.0, 0.0, 0.0], [21.275802, 10.435149, 6.600368], [36.689647, 12.773414, -2.438714], [6.937698, 36.11547600000001, 3.919102], [0.0, 0.0, 0.0]], 99: [[-0.197886, 1.011144, 2.862259], [1.6827, -0.085962, -6.520082], [7.96085, -0.395584, -18.75887], [-7.173747, 1.533488, 24.492029], [4.545798, 10.076018, 2.649374], [0.0, 0.0, 0.0], [-16.667757, 0.83749, 14.303908], [-0.664312, 1.358061, 25.352305], [0.685098, 3.157705, -2.234402], [0.0, 0.0, 0.0], [-3.7540010000000006, -0.163148, 18.778631000000004], [5.820179, -0.9690799999999999, -22.876057], [1.426262, 7.5892859999999995, 26.801681], [0.139836, 2.024047, -14.488937999999997], [0.0, 0.0, 0.0], [-32.987605, -1.514262, -4.544514], [-39.845623, 4.771768, -7.065708], [-11.39837, -19.424155, 2.598841], [0.0, 0.0, 0.0], [22.394228, 8.702663, 5.921297], [39.38168, 9.761454, -3.484783], [6.28535, 36.613491, 3.921616], [0.0, 0.0, 0.0]], 100: [[-0.175527, 1.008711, 2.903562], [2.82175, -0.320834, -6.689833], [5.672185, -0.66987, -16.36025], [-8.168086, 1.966626, 25.781190000000002], [5.738398, 11.623105, -1.260604], [0.0, 0.0, 0.0], [-14.753846, 0.266419, 4.124658], [-5.913076, 5.757193, 46.661037], [0.622534, 3.761566, -3.419229], [0.0, 0.0, 0.0], [-5.567852, 0.13786, 19.299395], [5.813414, -0.920732, -23.543168000000005], [1.366935, 7.404322, 26.944258], [0.436126, 1.81478, -15.300355], [0.0, 0.0, 0.0], [-31.838232000000005, -2.617663, -3.344349], [-39.502271, 3.069008, -6.607537], [-11.153251000000001, -21.866984, 2.896654], [0.0, 0.0, 0.0], [22.230329, 9.004601, 5.940726], [42.665955, 7.553623, -4.309235], [6.444171000000001, 35.846326, 3.9096910000000005], [0.0, 0.0, 0.0]], 101: [[-0.15515, 1.006396, 2.941296], [3.284014, -0.39873, -7.915878999999999], [3.787739, -0.764776, -13.379729], [-8.14867, 2.064839, 27.696171], [6.414419, 12.56798, -3.906809], [0.0, 0.0, 0.0], [-12.057267, 0.05288, -3.49376], [-7.842769999999999, 9.704537, 62.705380999999996], [-0.24067000000000002, 4.20987, -7.676087000000001], [0.0, 0.0, 0.0], [-5.69215, 0.17432200000000003, 21.153157], [5.765979, -0.8819920000000001, -24.142052], [0.109908, 5.926175, 26.830964], [0.202645, 1.787583, -15.907529], [0.0, 0.0, 0.0], [-32.477107, -3.165189, -2.337559], [-36.91115, 0.624171, -5.667356], [-10.47891, -23.129097000000005, 2.964289], [0.0, 0.0, 0.0], [23.239247, 9.484891, 5.836743], [43.544661, 6.947366, -4.491101], [7.700177, 33.6941, 3.894846], [0.0, 0.0, 0.0]], 102: [[-0.138673, 1.005858, 2.97924], [3.063046, 0.404652, -9.306266], [1.9632229999999997, -0.781551, -10.510964], [-6.733998, 1.522107, 30.322278000000004], [6.663791, 12.539073000000002, -6.677459], [0.0, 0.0, 0.0], [-8.728048000000001, 0.24369200000000002, -10.698092], [-6.055106, 9.795359, 73.411547], [-1.819565, -0.517684, -8.226386], [0.0, 0.0, 0.0], [-4.817827, -0.024011, 22.225474], [5.903368, -0.91605, -24.194592000000004], [-1.302168, 3.698548, 26.715827], [-0.150687, 1.716216, -15.428418000000002], [0.0, 0.0, 0.0], [-33.045413, -4.161708, -0.911011], [-34.927037, -2.018446, -4.896589], [-8.320106, -23.929068, 2.65072], [0.0, 0.0, 0.0], [23.688837, 9.666935, 5.423099], [44.772139, 5.232876, -5.047808], [8.925745, 33.179186, 4.102782], [0.0, 0.0, 0.0]], 103: [[-0.124819, 1.003521, 3.014834], [2.88994, 0.29630700000000004, -9.948171], [0.9520359999999999, -0.680168, -8.128904], [-6.377698, 1.438972, 32.15317400000001], [7.14869, 13.23517, -9.217376], [0.0, 0.0, 0.0], [-5.514965, 0.949873, -18.563692], [-3.762233, 8.149334, 76.842708], [-2.675633, -2.424787, -9.691652], [0.0, 0.0, 0.0], [-3.933357, -0.180217, 22.35431], [5.762645, -0.874926, -24.354506], [-2.13574, 2.159823, 26.869207], [-0.33267700000000006, 1.597538, -15.540458999999998], [0.0, 0.0, 0.0], [-32.512123, -4.742011, 0.270295], [-34.859085, -4.721896, -4.429361], [-5.774599, -24.707758, 2.235374], [0.0, 0.0, 0.0], [23.370793, 9.090994, 4.571508], [46.067323, 2.899468, -5.88541], [11.010734, 33.891682, 4.777895], [0.0, 0.0, 0.0]], 104: [[-0.112492, 0.999595, 3.049276], [2.426628, -0.10811, -10.402116], [1.019829, -0.464055, -5.681428], [-6.960654, 1.761821, 33.293857], [7.527708, 13.818098, -11.663807], [0.0, 0.0, 0.0], [-2.573841, 1.953579, -24.738783], [-1.11602, 5.187848, 72.647223], [-2.883895, -3.057483, -9.710634], [0.0, 0.0, 0.0], [-2.968614, -0.339849, 22.63112], [5.897087, -0.891951, -24.733469], [-2.6123730000000003, 0.489927, 26.95544], [-0.721948, 1.631945, -16.577612], [0.0, 0.0, 0.0], [-32.609018, -4.295902, 0.783584], [-34.92337, -6.979818, -4.109906], [-4.276128, -25.748566000000004, 2.063215], [0.0, 0.0, 0.0], [23.215144, 8.774059, 3.8003740000000006], [46.130983, 0.8441740000000001, -6.486239], [12.493943, 34.189188, 5.15755], [0.0, 0.0, 0.0]], 105: [[-0.102092, 0.993986, 3.084627], [2.068953, 0.080384, -10.928426], [0.786261, -0.229329, -2.802541], [-7.023062, 1.768202, 33.87505000000001], [7.736998999999999, 13.859053, -14.321308], [0.0, 0.0, 0.0], [-1.612083, 2.446549, -27.888761000000002], [2.925571, 1.009606, 62.747557], [-3.0809460000000004, -6.350947, -3.7207600000000003], [0.0, 0.0, 0.0], [-2.107835, -0.516541, 23.248352], [5.9568010000000005, -0.85785, -25.363508], [-3.106995, -1.411981, 27.066125], [-0.8882070000000001, 1.513337, -18.136093], [0.0, 0.0, 0.0], [-32.401403, -3.67754, 1.132737], [-35.335179, -8.879821, -3.901948], [-3.988688, -26.915433, 2.170416], [0.0, 0.0, 0.0], [23.153329, 7.889998, 2.74695], [46.217489, -0.393993, -6.820489], [12.785795, 33.697787, 5.048579], [0.0, 0.0, 0.0]], 106: [[-0.092388, 0.986173, 3.121371], [2.132943, 0.24731800000000004, -10.933388], [-0.11567199999999998, -0.067051, -0.690753], [-6.69197, 1.634591, 34.932945000000004], [7.949672, 14.021982, -16.652239], [0.0, 0.0, 0.0], [-1.342082, 2.415435, -26.963939], [5.172679, -0.750335, 47.353017], [-3.607447, -9.008372, 0.330351], [0.0, 0.0, 0.0], [-1.459488, -0.626372, 22.793451], [5.827657, -0.796501, -25.66369], [-3.294361, -2.610375, 27.32209], [-0.781687, 1.315844, -20.060086], [0.0, 0.0, 0.0], [-31.840882, -3.656021, 1.618547], [-36.308854, -10.1383, -3.8068549999999997], [-4.711645, -27.917967, 2.497411], [0.0, 0.0, 0.0], [22.777837, 6.834583, 1.821424], [46.570013, -2.20033, -7.463704], [12.624223, 33.838713, 4.963521], [0.0, 0.0, 0.0]], 107: [[-0.083305, 0.975886, 3.161801], [3.335765, -0.192025, -11.048879], [-2.476847, 0.189592, 1.998796], [-6.292131, 1.595892, 35.551202], [8.346934, 15.177762, -18.853026], [0.0, 0.0, 0.0], [-0.929026, 2.148665, -22.994986000000004], [3.6152470000000005, -0.011189, 31.272257], [-2.918855, -8.236527, 3.124565], [0.0, 0.0, 0.0], [-2.029559, -0.464011, 21.495904], [5.609277, -0.751193, -25.608864], [-2.873018, -2.396061, 27.872939], [-0.309634, 1.0842, -22.7524], [0.0, 0.0, 0.0], [-31.416575000000005, -3.37262, 1.562517], [-37.655795, -10.995485000000002, -3.738444], [-6.345473000000001, -28.273511, 2.983728], [0.0, 0.0, 0.0], [22.34954, 5.633629000000001, 1.11162], [46.860064, -4.326988, -8.329905], [12.481493, 33.627233, 4.889929], [0.0, 0.0, 0.0]], 108: [[-0.075836, 0.964128, 3.204777], [4.487922, -0.345884, -11.114737], [-4.628023, 0.618718, 5.147363000000001], [-5.85761, 1.474817, 35.666103], [8.667936, 16.118463, -21.523024], [0.0, 0.0, 0.0], [-0.40317700000000006, 1.70813, -17.606317], [-0.023152, 0.772543, 17.598791], [-1.173724, -5.396586000000001, 5.9921370000000005], [0.0, 0.0, 0.0], [-2.44712, -0.362207, 20.525322], [5.046065, -0.491928, -27.281633], [-2.040622, -2.03681, 29.029603], [-0.278186, 1.194353, -25.272814], [0.0, 0.0, 0.0], [-31.848764000000003, -2.79152, 1.188619], [-38.951744, -10.820415, -3.982048], [-8.233259, -28.349629, 3.511627], [0.0, 0.0, 0.0], [22.268852000000003, 4.916852, 0.791723], [46.251282, -7.570346999999999, -9.502928], [13.435553, 33.670366, 5.193719], [0.0, 0.0, 0.0]], 109: [[-0.069901, 0.950913, 3.25202], [4.698016, 0.022819, -11.075863], [-4.907968, 1.146657, 8.893607], [-5.830683, 1.395479, 35.325052], [8.654153, 15.564949, -24.027779], [0.0, 0.0, 0.0], [-0.29179, 1.170436, -12.142012], [-1.394406, 0.668112, 9.599652], [0.671108, -1.7029280000000002, 6.680738], [0.0, 0.0, 0.0], [-1.632984, -0.46792300000000003, 19.156428], [4.824512, -0.36792600000000003, -28.129653], [-2.122031, -2.137819, 28.841139999999996], [0.339478, 0.9366770000000001, -27.242128], [0.0, 0.0, 0.0], [-32.81166, -1.57407, 0.236959], [-39.760341, -8.838023, -4.612233], [-10.868598, -28.005517, 4.152054], [0.0, 0.0, 0.0], [22.424076, 3.461873, -0.016245], [45.111969, -10.273502, -10.169542], [14.181541, 32.506789, 5.179371], [0.0, 0.0, 0.0]], 110: [[-0.066379, 0.936476, 3.302041], [4.621678000000001, 0.399033, -10.781687], [-4.721623, 1.678197, 12.873692], [-5.562673, 1.198836, 34.939377], [8.688938, 14.879511, -27.134215], [0.0, 0.0, 0.0], [-0.05763200000000001, 0.823253, -8.421303], [-1.7354090000000002, 0.642333, 8.058238], [1.434609, -0.393931, 7.477497999999999], [0.0, 0.0, 0.0], [-0.499526, -0.601615, 17.973869], [4.609476, -0.22700000000000004, -29.383332], [-2.480276, -2.199828, 28.252831], [1.021801, 0.704619, -29.231601], [0.0, 0.0, 0.0], [-33.394223, 0.044538, -1.100627], [-40.647329, -5.568145, -5.605384], [-13.223482, -26.744726, 4.535067], [0.0, 0.0, 0.0], [22.507729, 1.444813, -1.128562], [44.436519, -12.562324, -10.734119], [13.927962, 30.904695999999998, 4.823134], [0.0, 0.0, 0.0]], 111: [[-0.067297, 0.924174, 3.349221], [2.071905, -0.244666, -10.996301], [-1.039531, 1.637504, 16.002885], [-5.861258, 1.387202, 37.407567], [8.566914, 13.835681, -28.961123], [0.0, 0.0, 0.0], [3.717031, 0.599164, -4.449344], [-2.064261, 0.611729, 6.535911], [1.119133, -1.669293, 8.853493], [0.0, 0.0, 0.0], [2.813616, -1.004376, 17.432843], [4.797527, -0.253883, -29.638245], [-2.135342, -1.771794, 27.905741], [0.953407, 1.030173, -32.574498], [0.0, 0.0, 0.0], [-35.301344, 0.795997, -2.0116530000000004], [-40.674578, -2.682835, -6.408302], [-13.684512, -24.710172000000004, 4.250973], [0.0, 0.0, 0.0], [22.70776, 1.198859, -1.185078], [43.081062, -14.038313, -10.848851], [12.38364, 29.723503999999995, 4.213018], [0.0, 0.0, 0.0]], 112: [[-0.068519, 0.921945, 3.38304], [1.458576, -0.8305750000000001, -10.661433], [-0.162768, 1.279074, 13.309478], [-5.389515, 1.441238, 46.710038], [7.123373, 9.51219, -23.811745], [0.0, 0.0, 0.0], [2.716387, 1.027558, -8.273487], [1.154036, 0.408146, 16.361737], [0.536529, -1.523967, 5.610116], [0.0, 0.0, 0.0], [2.669411, -0.971836, 17.252034], [5.711901, -0.59124, -27.861137], [-1.403185, -1.845141, 27.014929], [1.047389, 1.138473, -35.617335], [0.0, 0.0, 0.0], [-35.879065, 0.523033, -1.7321420000000003], [-42.740941, -1.8217550000000002, -6.942756], [-10.46247, -22.149273, 3.156629], [0.0, 0.0, 0.0], [23.179864, 2.760338, -0.49046300000000004], [43.398164, -11.004198, -9.811597], [6.67236, 26.936062000000003, 2.458309], [0.0, 0.0, 0.0]], 113: [[-0.071747, 0.927326, 3.411603], [1.007019, 0.568397, -10.866972], [1.770232, 0.52839, 6.617484000000001], [-5.439658, 1.199685, 60.778618], [4.782911, 4.597098, -15.683712], [0.0, 0.0, 0.0], [3.501029, 0.84494, -6.371718], [1.08336, 0.387852, 16.906321], [-0.481515, -3.031288, 4.035168], [0.0, 0.0, 0.0], [1.678398, -0.936625, 18.085608], [6.599497, -0.839731, -27.198785], [-0.598203, -2.2498820000000004, 26.18021], [0.46567100000000006, 1.581536, -36.982154], [0.0, 0.0, 0.0], [-34.276893, -1.522885, -0.361812], [-44.476099000000005, -0.622497, -7.466252], [-10.201243, -21.082421, 2.915636], [0.0, 0.0, 0.0], [22.530293, 4.47961, 0.661926], [45.465919, -9.670565, -9.831908], [3.618829, 28.099398, 1.906502], [0.0, 0.0, 0.0]], 114: [[-0.075426, 0.931312, 3.434236], [1.572756, 2.308373, -10.520519], [3.024207, -0.01156, -0.604529], [-6.241206, 1.1236670000000002, 72.002225], [3.533005, 1.86933, -13.532817], [0.0, 0.0, 0.0], [3.430112, 0.46960999999999997, -3.477268], [0.12120900000000001, 0.4992789999999999, 14.468484000000002], [-0.980868, -4.154683, 4.202264], [0.0, 0.0, 0.0], [0.22550299999999998, -0.826496, 18.789444], [6.680823, -0.803419, -27.614179], [-0.4586180000000001, -1.637523, 26.100882], [0.765487, 1.768484, -38.460676], [0.0, 0.0, 0.0], [-33.16717, -2.689204, -0.178987], [-42.201908, -0.264473, -7.004273], [-13.616282, -21.893286, 3.622663], [0.0, 0.0, 0.0], [22.845596, 3.8689629999999995, 0.785167], [45.362984, -8.746297, -9.602199], [6.689656, 31.93325, 3.140421], [0.0, 0.0, 0.0]], 115: [[-0.074822, 0.937139, 3.454033], [2.030842, 2.804349, -10.12804], [3.5506210000000005, -0.419819, -7.168128], [-7.544713000000001, 2.235439, 79.491145], [3.000572, 2.520104, -9.542604], [0.0, 0.0, 0.0], [2.695685, 0.247679, -1.9022830000000002], [0.299561, 0.408362, 13.30977], [-1.074507, -4.2853580000000004, 4.011851], [0.0, 0.0, 0.0], [-0.364858, -0.757968, 18.810093], [6.631216, -0.8248480000000001, -27.127401], [-0.367496, -0.537494, 25.616463], [1.163117, 1.859303, -39.411982], [0.0, 0.0, 0.0], [-34.276047, -3.080505, -0.646864], [-39.61886, -0.265501, -6.345816], [-13.892396, -22.896987, 3.780156], [0.0, 0.0, 0.0], [22.650786, 4.678462, 1.65569], [44.146636, -7.2145280000000005, -8.87037], [10.546872, 35.348829, 4.779345], [0.0, 0.0, 0.0]], 116: [[-0.073851, 0.941568, 3.471714], [2.598587, 1.443119, -10.188881], [4.017101, -0.591471, -10.777488], [-8.553685, 4.269439, 80.276359], [3.564419, 5.015341, -7.194719], [0.0, 0.0, 0.0], [2.099412, -0.050298, 0.48342299999999994], [-0.022047000000000004, 0.444719, 11.369314], [-0.353282, -2.608441, 3.769232], [0.0, 0.0, 0.0], [-0.9709790000000001, -0.632598, 19.093015], [6.29438, -0.75159, -27.21896], [0.297921, 1.640397, 25.857746], [1.74515, 1.856633, -40.372049], [0.0, 0.0, 0.0], [-34.098543, -3.229938, -1.099472], [-40.065618, -0.4491, -6.207365], [-11.238659, -24.027087, 3.382827], [0.0, 0.0, 0.0], [21.765293, 5.796267, 2.67016], [44.004174, -4.449385, -8.035835], [11.208477, 37.438198, 5.432883], [0.0, 0.0, 0.0]], 117: [[-0.073781, 0.944544, 3.48613], [2.744517, 0.03017, -9.993184], [4.877054, -0.552017, -12.183152], [-8.982358, 5.092162, 73.67581400000002], [5.252614000000001, 9.052966, -7.465733000000001], [0.0, 0.0, 0.0], [2.13979, -0.223612, 1.941567], [-0.452129, 0.495438, 9.954441], [0.22859899999999997, -1.221703, 3.530978], [0.0, 0.0, 0.0], [-1.516716, -0.49324, 18.514165], [6.37327, -0.8973769999999999, -25.876154], [0.844511, 3.357189, 25.087637], [2.714105, 1.562294, -40.054974], [0.0, 0.0, 0.0], [-34.448004, -2.777297, -1.899487], [-39.989839, -0.775114, -5.895087], [-9.921737, -25.565134, 3.285042], [0.0, 0.0, 0.0], [21.319701, 6.13001, 3.038698], [43.234916, -0.254562, -6.509853], [10.611666, 37.955626, 5.368663], [0.0, 0.0, 0.0]], 118: [[-0.071344, 0.946662, 3.498011], [3.080111, -0.931536, -9.73558], [4.551375, -0.534642, -11.389882], [-8.202379, 4.023337, 60.999976000000004], [5.326871, 9.498887, -6.619695], [0.0, 0.0, 0.0], [1.560094, -0.444658, 4.036145], [-0.622757, 0.396365, 7.015505], [0.864016, 0.059102, 3.821747], [0.0, 0.0, 0.0], [-1.745437, -0.426223, 18.496054], [6.075665, -0.828449, -25.932335], [1.0674470000000003, 3.9350310000000004, 24.956245], [2.859946, 1.533795, -40.551085], [0.0, 0.0, 0.0], [-34.538875, -2.494435, -2.103102], [-39.902327, -1.383648, -5.686113], [-9.661523, -26.562531, 3.363189], [0.0, 0.0, 0.0], [21.158785, 6.733091, 3.348591], [42.51688800000001, 3.2497180000000006, -5.262157], [8.83502, 38.470234, 4.834823], [0.0, 0.0, 0.0]], 119: [[-0.068135, 0.947793, 3.50922], [3.0050500000000007, -1.904187, -9.476121], [3.773778, -0.377538, -7.357178000000001], [-5.95587, 1.9634880000000001, 43.351421], [4.3778120000000005, 7.508345, -6.096957], [0.0, 0.0, 0.0], [1.484366, -0.636003, 5.877571], [-0.957093, 0.320871, 4.393372], [1.530963, 1.309794, 4.26207], [0.0, 0.0, 0.0], [-1.353914, -0.441516, 17.971493], [5.733196, -0.791372, -25.524395], [1.140472, 4.039968, 24.77797], [2.72806, 1.568098, -41.115681], [0.0, 0.0, 0.0], [-34.071048, -2.862464, -1.573667], [-40.058578, -1.900706, -5.602486], [-10.615129, -27.056264, 3.672745], [0.0, 0.0, 0.0], [20.543857, 7.5550109999999995, 3.6418660000000003], [42.579706, 5.788773000000001, -4.372033], [6.603100000000001, 39.422726, 4.172975], [0.0, 0.0, 0.0]], 120: [[-0.064008, 0.947932, 3.521828], [2.616317, -2.795925, -9.69484], [2.721616, -0.010638, -0.909599], [-2.902459, 0.327771, 25.439252], [3.242325, 5.3471210000000005, -4.932855], [0.0, 0.0, 0.0], [1.636163, -0.8994659999999999, 8.362912], [-1.1823050000000002, 0.21082, 1.694862], [2.043504, 2.34745, 4.418383], [0.0, 0.0, 0.0], [-0.713114, -0.49411000000000005, 17.443644], [5.483907, -0.773757, -25.134927], [1.129845, 3.576107, 24.624297], [2.833652, 1.386837, -41.062844000000005], [0.0, 0.0, 0.0], [-33.672074, -3.135945, -0.885196], [-40.462615, -2.277043, -5.674519], [-11.779606, -27.000934, 3.984926], [0.0, 0.0, 0.0], [19.866582, 8.442168, 3.8242310000000006], [42.52712, 7.456987, -3.718566], [5.590438, 39.783506, 3.767977], [0.0, 0.0, 0.0]], 121: [[-0.05971, 0.946741, 3.534176], [1.744156, -3.2586510000000004, -9.873882], [2.348715, 0.379573, 4.329905], [-0.9514800000000001, -0.17385000000000003, 13.686445], [2.415992, 4.853595, -0.961325], [0.0, 0.0, 0.0], [1.9712510000000003, -1.191015, 10.898689], [-0.86222, 0.03946, -1.053802], [2.452992, 3.029694, 4.815543], [0.0, 0.0, 0.0], [-0.002733, -0.559347, 16.603963], [5.539795, -0.811027, -24.940314], [1.075662, 2.410444, 24.389057], [3.196797, 0.9770100000000002, -40.208751], [0.0, 0.0, 0.0], [-33.722949, -3.042957, -0.330354], [-41.056066, -2.543496, -5.844354], [-12.460974, -26.575575, 4.139748], [0.0, 0.0, 0.0], [19.562524, 9.081741, 3.782753], [41.983033, 8.331928, -3.267399], [6.075098, 39.654149, 3.7656570000000005], [0.0, 0.0, 0.0]], 122: [[-0.055417, 0.944964, 3.545312], [0.917298, -3.143992, -9.77948], [2.345281, 0.662745, 8.002884], [-0.22554, -0.21477700000000002, 8.183681], [2.742522, 5.630380000000001, -0.768884], [0.0, 0.0, 0.0], [2.395139, -1.37941, 12.151477], [-0.726544, -0.034314, -2.402442], [2.566306, 3.21805, 4.971299000000001], [0.0, 0.0, 0.0], [0.122256, -0.560286, 15.751677000000003], [5.768202, -0.841744, -25.198391000000004], [1.042146, 0.779922, 24.470325], [3.820569, 0.44845899999999994, -39.77433400000001], [0.0, 0.0, 0.0], [-33.472015, -2.429148, -0.052774], [-42.18466, -3.287868, -5.975611], [-12.58266, -26.158989, 4.180404], [0.0, 0.0, 0.0], [19.462665, 8.864634, 3.269042], [41.972835, 8.776256, -3.015868], [6.791438, 39.532371000000005, 3.8571690000000003], [0.0, 0.0, 0.0]], 123: [[-0.052001, 0.946253, 3.554584], [0.692194, -2.65732, -9.683673], [2.841494, 0.7913530000000001, 10.228406], [-0.662021, -0.041152, 4.425386], [1.9962690000000003, 4.708923, 1.268203], [0.0, 0.0, 0.0], [2.699993, -1.450382, 12.350408], [-1.281949, 0.025057, -2.167927], [2.379233, 2.873305, 4.904519], [0.0, 0.0, 0.0], [-0.769964, -0.444502, 14.625143], [6.013339, -0.920378, -24.909702000000003], [1.1211040000000003, -0.457245, 24.535506], [4.092098, 0.174477, -39.621368], [0.0, 0.0, 0.0], [-33.06931, -2.326061, 0.33726800000000007], [-43.175824, -4.132885, -5.955873], [-11.981573, -25.738533, 4.019897], [0.0, 0.0, 0.0], [20.033297, 8.171595, 2.63098], [43.441643, 7.798049, -3.350984], [5.507574, 39.676172, 3.382019], [0.0, 0.0, 0.0]], 124: [[-0.047393, 0.94818, 3.56238], [0.694181, -2.07869, -9.680253], [2.445575, 0.884106, 11.248186], [-0.46868100000000007, -0.039005, 3.1961830000000004], [1.72628, 4.18818, 1.361832], [0.0, 0.0, 0.0], [2.567766, -1.454848, 12.39524], [-1.514737, 0.083311, -1.474823], [1.9915660000000002, 2.490649, 4.107709], [0.0, 0.0, 0.0], [-1.117915, -0.400896, 13.87929], [5.906562, -0.866921, -25.081525000000003], [0.681264, -0.885605, 24.634796], [4.084665, 0.143614, -39.445452], [0.0, 0.0, 0.0], [-32.29661, -2.36627, 0.441648], [-43.724242, -4.759947, -5.825397], [-12.048133, -25.325264, 3.975541], [0.0, 0.0, 0.0], [19.97016, 6.995741, 1.9808299999999999], [45.482195, 5.471344, -4.31852], [6.390364, 39.231565, 3.6835090000000004], [0.0, 0.0, 0.0]], 125: [[-0.04182, 0.947589, 3.568749], [0.527432, -2.017871, -9.841037], [2.012487, 1.030391, 12.630868], [-0.095997, -0.042764, 1.628345], [1.774067, 4.335638, 1.488388], [0.0, 0.0, 0.0], [2.498772, -1.434661, 12.274046], [-1.756193, 0.17416, -0.24682599999999996], [1.753821, 2.462844, 3.213282], [0.0, 0.0, 0.0], [-0.915984, -0.422206, 13.843227], [5.725536000000001, -0.809389, -25.244873], [0.382325, -0.832362, 24.592017], [3.786867, 0.326469, -39.814971], [0.0, 0.0, 0.0], [-32.570215, -2.275104, 0.30804], [-43.313262, -5.023117, -5.674611], [-12.138163, -25.48657, 4.014585], [0.0, 0.0, 0.0], [20.067838, 6.432691, 1.704821], [46.524397, 2.888568, -5.356906000000001], [7.419205, 37.716906, 3.884748], [0.0, 0.0, 0.0]], 126: [[-0.03668, 0.946853, 3.573636], [0.403223, -1.828303, -9.902677], [1.9522830000000002, 1.073253, 13.169329], [-0.24480699999999997, -0.010643, 1.229748], [1.694095, 4.126369, 1.354737], [0.0, 0.0, 0.0], [2.366897, -1.434312, 12.353624], [-1.884045, 0.219682, 0.304446], [1.569493, 2.336014, 2.700607], [0.0, 0.0, 0.0], [-0.887469, -0.42736, 13.776349], [5.570496, -0.767781, -25.261351], [0.271475, -0.902322, 24.577549], [3.568575, 0.462226, -40.17278], [0.0, 0.0, 0.0], [-33.092839, -2.250785, 0.235492], [-42.88535, -5.35836, -5.510907], [-11.643275, -25.285159, 3.85842], [0.0, 0.0, 0.0], [20.292532, 6.285607, 1.633725], [46.416214, 0.696616, -6.137212], [9.046153, 35.004849, 4.043925], [0.0, 0.0, 0.0]], 127: [[-0.032234, 0.946753, 3.577872], [0.178087, -1.335018, -9.815218], [2.002688, 1.028616, 12.913037000000001], [-0.343126, -0.014460999999999998, 1.742053], [1.615247, 3.8068350000000004, 0.925335], [0.0, 0.0, 0.0], [2.324157, -1.491963, 12.763575], [-1.813882, 0.18967, -0.12186200000000001], [1.417111, 2.020474, 2.656177], [0.0, 0.0, 0.0], [-0.942631, -0.422401, 13.438537], [5.561795, -0.755888, -25.278332], [0.332784, -1.08656, 24.617816], [3.306182, 0.599763, -40.270309], [0.0, 0.0, 0.0], [-32.794045, -2.498451, 0.366355], [-43.657316, -5.780143, -5.43461], [-10.534591000000002, -24.330032, 3.458922], [0.0, 0.0, 0.0], [20.02748, 6.326266, 1.708801], [46.870472, -1.291666, -6.935399], [9.34439, 31.534353000000003, 3.669801], [0.0, 0.0, 0.0]], 128: [[-0.027915, 0.947029, 3.58146], [-0.086408, -0.904632, -9.731621], [1.9417369999999998, 0.9972850000000001, 12.646203000000002], [-0.208216, -0.049493, 2.254625], [1.60601, 3.637927, 0.532025], [0.0, 0.0, 0.0], [2.453296, -1.5439660000000002, 12.959381], [-1.930021, 0.197731, -0.227519], [1.232166, 1.657516, 2.547684], [0.0, 0.0, 0.0], [-0.9762870000000001, -0.424297, 13.299887], [5.665819000000001, -0.765935, -25.422083], [0.444715, -1.247336, 24.537492], [3.110078, 0.702839, -40.276684], [0.0, 0.0, 0.0], [-32.54399, -2.626734, 0.40525], [-44.177691, -5.823179, -5.445123], [-10.159253, -23.62818, 3.268008], [0.0, 0.0, 0.0], [20.088521, 5.939321, 1.531572], [47.554279, -3.265668, -7.770785], [7.984806000000001, 28.910559, 3.005393], [0.0, 0.0, 0.0]], 129: [[-0.024062, 0.94671, 3.58462], [-0.248275, -0.634896, -9.694844], [1.8852290000000003, 0.999071, 12.728903], [-0.203233, -0.051419, 2.280452], [1.579256, 3.460467, 0.223368], [0.0, 0.0, 0.0], [2.376897, -1.556023, 13.062411], [-1.8788639999999999, 0.200404, -0.099253], [1.0899, 1.459399, 2.307086], [0.0, 0.0, 0.0], [-1.12296, -0.412127, 13.170664000000002], [5.829582, -0.801144, -25.413784], [0.509836, -1.4061260000000002, 24.501370000000005], [3.238016, 0.661687, -40.412653], [0.0, 0.0, 0.0], [-32.605659, -2.5173550000000002, 0.318676], [-44.037876, -5.668049, -5.480954], [-10.397375, -23.416127, 3.284484], [0.0, 0.0, 0.0], [20.49708, 5.166436, 1.0935540000000001], [48.01138, -5.06385, -8.516116000000002], [6.173501, 27.645519, 2.449939], [0.0, 0.0, 0.0]], 130: [[-0.020342, 0.946252, 3.588073], [-0.362243, -0.503271, -9.746111], [1.754371, 1.02857, 13.007331000000002], [-0.156301, -0.053888, 2.159828], [1.575155, 3.3927370000000003, 0.075879], [0.0, 0.0, 0.0], [2.266018, -1.568444, 13.238843], [-1.884102, 0.20993, 0.049322], [1.017286, 1.410904, 2.085844], [0.0, 0.0, 0.0], [-1.204653, -0.404426, 13.05377], [5.9472, -0.82815, -25.424888], [0.427828, -1.797262, 24.669490000000003], [3.519675, 0.49268100000000004, -40.401857], [0.0, 0.0, 0.0], [-32.520495, -2.070133, 0.150784], [-44.013774, -5.374816, -5.634591], [-10.636662, -23.360794, 3.341064], [0.0, 0.0, 0.0], [20.620736, 4.314899, 0.5387], [48.168809, -6.387722, -9.039603000000001], [5.579073000000001, 27.382263, 2.282859], [0.0, 0.0, 0.0]], 131: [[-0.016591, 0.946412, 3.590416], [-0.487212, -0.476437, -9.709154], [1.553175, 1.043418, 12.962664000000002], [-0.034584000000000004, -0.07645, 2.341331], [1.654373, 3.494307, -0.090781], [0.0, 0.0, 0.0], [2.247985, -1.559184, 13.161274], [-2.042487, 0.24657, 0.377121], [0.977438, 1.398919, 1.934513], [0.0, 0.0, 0.0], [-1.119251, -0.408752, 12.900195], [6.080571, -0.845122, -25.600672], [0.168709, -2.3875420000000003, 24.864938], [4.415165, 0.006683, -40.2101], [0.0, 0.0, 0.0], [-32.607268, -1.572693, 0.051094], [-43.96205400000001, -5.017005, -5.843028], [-10.34293, -23.098836000000002, 3.262127], [0.0, 0.0, 0.0], [20.677427, 3.649224, 0.016502], [47.820455, -7.360785, -9.311781], [6.097931, 27.926663, 2.426626], [0.0, 0.0, 0.0]]}


In [12]:
def get_jointpos_from_bvh(filepath: Path, return_anim: bool = False) -> torch.Tensor:
    """Load a BVH file and convert it to HML3D_abs format."""
    assert Path(filepath).is_file(), "BVH file not found"

    animation = BVH.load(filepath)

    # TODO: Define all assumptions about the input BVH file as `asserts`
    assert animation.positions.shape[1] == 22, "Incorrect number of joints in BVH file"

    # Get global joint positions
    joint_positions = Animation.positions_global(animation)

    # Reorder joints to undo the reordering of Joints2BVHConvertor
    joint_positions = joint_positions[:, joints2bvh.re_order_inv]

    if return_anim:
        return joint_positions, animation
    else:
        return joint_positions


def unpack_motion(
    packed_motion: dict[int, list]
) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Unpack the packed motion input and return the joint positions and mask.

    Packed Motion Format:
    - A dictionary mapping frame indices to packed poses
    - A packed pose contains the root position followed by all 22 joint rotations
    - Values stored as `nan` indicate sparse keyframes and are converted to a joint mask
    """
    packed_motion = {int(k): np.array(v) for k, v in packed_motion.items()}

    # We actually have 22 joints, but are piggybacking the root pos in the first index.
    assert next(iter(packed_motion.values())).shape[0] == 23, "Expected 22 joints + 1"

    input_frames = np.max(list(packed_motion.keys()))
    motion = np.zeros((input_frames + 1, 22 + 1, 3))
    joint_mask = np.zeros((input_frames + 1, 22 + 1, 1), dtype=bool)
    for frame, pose in packed_motion.items():
        _pose_mask = ~np.isnan(pose)
        motion[frame, _pose_mask] = pose[_pose_mask]
        joint_mask[frame] = _pose_mask.all(axis=-1).reshape(-1, 1)

    # Extract root position and mask
    root_pos = motion[:, 0].copy()
    root_mask = joint_mask[:, 0].copy()
    rotations = motion[:, 1:]
    joint_mask = joint_mask[:, 1:]

    # Theoretically, this should always hold true, otherwise we would
    # need separate joint and feature masks (which is possible).
    assert np.all(
        np.equal(root_mask, joint_mask[:, 0])
    ), "Root pos mask does not match root rot mask"

    return root_pos, rotations, joint_mask

def unpacked_motion_to_jointpos(
    root_pos: np.ndarray,
    rotations: np.ndarray,
    return_anim: bool = False,
) -> torch.Tensor:
    """Convert the unpacked motion to joint positions using the BVH template and
    Forward Kinematics.

    This is based on the following list of assumptions:
    - The skeleton is the same as in `template.bvh` (22 joints)
        - This is used to get `parents`, `offsets` and `names`
    - Frametime is 1/20s (as per HML3D dataset)
    - Rotation order is XYZ
    """

    # HACK: Hardcoded for now
    # TODO: Why? Why is this needed when Maya says it's xyz?!
    ORDER = "zyx"
    input_frames = rotations.shape[0]

    # Using the BVH template to get offsets, orients, parents and names
    template_path = Path(BVH.__file__).parent / "data" / "template.bvh"
    template_anim = BVH.load(template_path)

    # NOTE: `positions` and `rotations` are both local to the parent joint
    anim = template_anim.copy()
    anim.frametime = 1.0 / 20.0
    anim.rotations = Quaternions.from_euler(np.radians(rotations), order=ORDER)
    anim.positions = anim.offsets[np.newaxis].repeat(input_frames, axis=0)
    anim.positions[:, 0] = root_pos

    joint_positions = Animation.positions_global(anim)

    # TODO: Assure the original BVH ordering in the Maya/Blender Export
    #       Also investigate the reason for this whole (re)ordering,
    #       do we really need hardcoded indices here?

    # Reorder joints to undo the reordering of Joints2BVHConvertor
    joint_positions = joint_positions[:, joints2bvh.re_order_inv]

    if return_anim:
        return joint_positions, anim
    else:
        return joint_positions

In [13]:
pm_root_pos, pm_rotations, _ = unpack_motion(packed_motion)
pm_jointpos, pm_anim = unpacked_motion_to_jointpos(pm_root_pos, pm_rotations, return_anim=True)
pm_anim: Animation.Animation


In [14]:
# CHECKS
allclose_joints = np.allclose(bvh_jointpos, pm_jointpos)

# Check positions, rotations, offsets, orients, parents, names, frametime
allclose_anim_pos = np.allclose(bvh_anim.positions, pm_anim.positions)
allclose_anim_rot = np.allclose(bvh_anim.rotations, pm_anim.rotations)
allclose_anim_offset = np.allclose(bvh_anim.offsets, pm_anim.offsets)
allclose_anim_orient = np.allclose(bvh_anim.orients, pm_anim.orients)
same_anim_parents = np.all(bvh_anim.parents == pm_anim.parents)
same_anim_names = bvh_anim.names == pm_anim.names
same_anim_frametime = bvh_anim.frametime == pm_anim.frametime

print("--------- Jointpos ---------")
print(f"{allclose_joints=}")
print("----------- Anim -----------")
print(f"{allclose_anim_pos=}")
print(f"{allclose_anim_rot=}")       # ! THIS IS THE CULPRIT
print(f"{allclose_anim_offset=}")
print(f"{allclose_anim_orient=}")
print(f"{same_anim_parents=}")
print(f"{same_anim_names=}")
print(f"{same_anim_frametime=}")
print("----------------------------")


--------- Jointpos ---------
allclose_joints=True
----------- Anim -----------
allclose_anim_pos=True
allclose_anim_rot=True
allclose_anim_offset=True
allclose_anim_orient=True
same_anim_parents=True
same_anim_names=True
same_anim_frametime=True
----------------------------


In [7]:
bvh_anim_quats = bvh_anim.rotations.qs[0]
bvh_anim_quats[0]

array([ 0.999192, -0.035982,  0.017344,  0.0044  ])

In [8]:
pm_anim_quats = pm_anim.rotations.qs[0]
pm_anim_quats[0]

array([ 0.999192,  0.0044  ,  0.017344, -0.035982])

`pm_anim` seems to be flipped somehow?

In [9]:
pm_euler_quats = R.from_euler("xyz", pm_rotations[0], degrees=True).as_quat()
pm_euler_quats[0, [3, 0, 1, 2]]

array([ 0.999192,  0.0044  ,  0.017344, -0.035982])

In [10]:
Quaternions.from_euler(np.deg2rad(pm_rotations)[0], order="zyx").qs[0]

array([ 0.999192, -0.035982,  0.017344,  0.0044  ])

In [11]:
Quaternions.from_euler(np.deg2rad(pm_rotations)[0], world=True).qs[0]

array([ 0.999192,  0.0044  ,  0.017344, -0.035982])

---
---
---


In [60]:
res = np.load("save/results/condmdi_random_joints/MoLab000750000__benchmark_sparse_pos_rot_T=30_CI=0_CRG=0_KGP=1.0_seed10_A_person_walking_over_stones/results.npy", allow_pickle=True).item()

In [61]:
obs_motion, motion = res["observed_motion"], res["motion"]
print("Raw observed and sampled first frame of root pos:")
obs_motion[0, 0, 0], motion[0, 0, 0, 0]

Raw observed and sampled first frame of root pos:


(array([0.      , 1.912155, 0.      ], dtype=float32),
 array([ 0.01607 ,  1.836665, -0.008328], dtype=float32))

In [62]:
from data_loaders.humanml.scripts.motion_process import postprocess_motion


obs_motion_pp = postprocess_motion(obs_motion, res["pre_y"], res["pre_xz"], res["pre_rot"])
motion_pp = res["postprocessed_motion"]
print("Postprocessed observed and sampled first frame of root pos:")
obs_motion_pp[0, 0, 0], motion_pp[0][0, 0]

Postprocessed observed and sampled first frame of root pos:


(array([0.      , 0.991798, 0.      ], dtype=float32),
 array([ 0.016095,  0.916308, -0.00828 ], dtype=float32))

In [63]:
obs_motion.shape

(3, 196, 22, 3)

In [64]:
np.random.uniform(-2, 2, (1 + 1, 3))

array([[ 1.480455, -0.526508,  1.845469],
       [ 0.422536, -0.456124,  1.798405]])

In [69]:
np.random.normal([0.1, 0.95, 0.05], [1.2, 0.32, 2.4], (1 + 1, 3))


array([[-0.283788,  1.080646, -1.843474],
       [-0.099977,  0.736444, -2.277908]])